In [4]:
import json
from typing import List, Dict, Optional

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate


# === Configuration ===
BASE_URL = "192.168.0.15:11434"
EMBED_MODEL = "hellord/e5-mistral-7b-instruct:Q4_0"


# LLM initialization
llm = ChatOllama(
    model="phi4:latest",
    temperature=0,
    base_url=BASE_URL,
)

# Embeddings initialization
embeddings = OllamaEmbeddings(
    model=EMBED_MODEL,
    base_url=BASE_URL,
)

In [7]:
import logging
import time
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Console handler
console_handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

# File handler
file_handler = logging.FileHandler('application.log')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
import json

import re
from typing import List, Dict, Optional
from langchain_community.document_loaders import PyPDFLoader
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_community.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from typing import List, Optional, Tuple
from pathlib import Path
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
import pandas as pd
import util
from dotenv import load_dotenv
import os
load_dotenv()
# === Configuration ===
BASE_URL = os.getenv("BASE_URL")
EMBED_MODEL = os.getenv("EMBED_MODEL")
MODEL = os.getenv("MODEL")
logger.info(f"Loading llm Model- {MODEL} embedd model- {EMBED_MODEL}")
# LLM initialization 
llm = ChatOllama(
    model=MODEL,
    temperature=0,
    base_url=BASE_URL,
)

# Embeddings initialization
embeddings = OllamaEmbeddings(
    model=EMBED_MODEL,
    base_url=BASE_URL,
)

def build_vectorstore(
    pdf_path: str,
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
    loader_type: str = "pypdf",
) -> Tuple[FAISS, List[str], BM25Retriever]:
    """
    Builds and returns vectorstore, text chunks, and BM25 retriever.
    """
    # Validate file exists
    pdf_file = Path(pdf_path)
    if not pdf_file.exists():
        raise FileNotFoundError(f"PDF not found: {pdf_path}")

    # Load document
    loader = {
        "pypdf": PyPDFLoader,
        "unstructured": UnstructuredPDFLoader
    }.get(loader_type)

    if not loader:
        raise ValueError(f"Unsupported loader_type: {loader_type}")
    docs = loader(str(pdf_file)).load()   
    # Split into chunks
    splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, keep_separator=True)
    split_docs = splitter.split_documents(docs)

    # Create FAISS index
    faiss_index = FAISS.from_documents(split_docs, embeddings)

    # Create BM25 retriever
    bm25_retriever = BM25Retriever.from_documents(split_docs)
    bm25_retriever.k = 5

    return faiss_index, [doc.page_content for doc in split_docs], bm25_retriever


def retrieve_chunks(
    faiss_index: FAISS,
    bm25_retriever: BM25Retriever,
    query: str,
    mode: str = "keyword",
    k: int = 5
) -> List[str]:
    """
    Retrieves matching chunks based on the mode.
    """
    if mode == "similarity":
        retriever = faiss_index.as_retriever(search_type="similarity", search_kwargs={"k": k})
    elif mode == "keyword":
        bm25_retriever.k = k
        retriever = bm25_retriever
    elif mode == "hybrid":
        sim_retriever = faiss_index.as_retriever(search_type="similarity", search_kwargs={"k": k})
        bm25_retriever.k = k

        class HybridRetriever:
            def __init__(self):
                self.sim = sim_retriever
                self.bm = bm25_retriever
                self.k = k

            def get_relevant_documents(self, query: str) -> List[str]:
                sim_docs = self.sim.get_relevant_documents(query)
                bm_docs = self.bm.get_relevant_documents(query)
                seen, combined = set(), []
                for doc in sim_docs + bm_docs:
                    if doc.page_content not in seen:
                        seen.add(doc.page_content)
                        combined.append(doc)
                    if len(combined) >= self.k:
                        break
                return combined

        retriever = HybridRetriever()
    else:
        raise ValueError("mode must be one of: similarity, keyword, hybrid")

    results = retriever.get_relevant_documents(query)
    return results


2025-07-23 14:44:03,045 - INFO - Loading llm Model- phi4:latest embedd model- mxbai-embed-large:latest
2025-07-23 14:44:03,045 - INFO - Loading llm Model- phi4:latest embedd model- mxbai-embed-large:latest
2025-07-23 14:44:03,045 - INFO - Loading llm Model- phi4:latest embedd model- mxbai-embed-large:latest
2025-07-23 14:44:03,045 - INFO - Loading llm Model- phi4:latest embedd model- mxbai-embed-large:latest


In [9]:
# Example input values
pdf_path = "pdfs/Losec Control SmPC.pdf"  # Replace with your actual PDF file path
query = "List of Excipient in '6. PHARMACEUTICAL PARTICULARS' , List of excipients, PHARMACEUTICAL PARTICULARS"  # Replace with your query
mode = "keyword"  # Can be "similarity", "keyword", or "hybrid"
k = 5

try:
    # Build vectorstore
    print("Building vectorstore...")
    faiss_index, chunks, bm25_retriever = build_vectorstore(pdf_path)

    # Retrieve chunks
    print(f"\nRetrieving top {k} chunks for query: '{query}' using mode: {mode}")
    results = retrieve_chunks(faiss_index, bm25_retriever, query, mode=mode, k=k)

    # Print results
    print("\nRetrieved Chunks:")
    for i, doc in enumerate(results, 1):
        print(f"\n--- Chunk {i} ---")
        print(doc.page_content)

except Exception as e:
    print(f"Error: {e}")

Building vectorstore...

Retrieving top 5 chunks for query: 'List of Excipient in '6. PHARMACEUTICAL PARTICULARS' , List of excipients, PHARMACEUTICAL PARTICULARS' using mode: keyword

Retrieved Chunks:

--- Chunk 1 ---
BCH23004 8/9  
 
Excretion 
The plasma elimination half -life of omeprazole is usually shorter than one hour both after single and  repeated 
oral once-daily dosing. Omeprazole is completely eliminated from plasma betwee n doses with no tendency for 
accumulation during o nce-daily administration. Almost 80% of an oral d ose of omeprazole is  excreted as 
metabolites in the urine, the remainder in the faeces, primarily originating from bile secretion. 
 
The AUC of omeprazole increases with repeated administration. This increase i s dose-dependent and results in 
a non-linear dose-AUC relationship after repeated administration. This time- and dose- dependency is due to a 
decrease of first pass metabolism and systemic clearance proba bly caused by an inhibition of the  

special_precautions_for_storage


In [1]:
import pandas as pd
import util
import re
special_precautions_for_storage = [
                "Do not store above 25°C.",
                "Store in the original package in order to protect from moisture ok fine."
            ]   
storage_codes = pd.read_csv("refrence files/100000073344.csv",keep_default_na=False)
matched, unmatched = util.get_storage_code(storage_codes,special_precautions_for_storage,language="en")
matched

['Do not store above 25°C. (100000073410)',
 'store in the original package (100000073421)',
 'in order to protect from moisture (100000073427)']

In [43]:
import pandas as pd

df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
print(df)
# Apply a function to each column (axis=0) to calculate the sum
column_sums = df.apply(lambda x: x.sum(), axis=0)
print("Column Sums:\n", column_sums)

# # Apply a function to each row (axis=1) to calculate the product
# row_products = df.apply(lambda x: x['A'] * x['B'], axis=1)
# print("\nRow Products:\n", row_products)

   A  B
0  1  4
1  2  5
2  3  6
Column Sums:
 A     6
B    15
dtype: int64


In [45]:
type(column_sums)

pandas.core.series.Series

In [18]:
text = '''{
    "shelf_life_time_period": "3 years",
    "special_precautions_for_storage": [
        "Do not store above 30° C.",
        "Bottle: Keep the container tightly closed in order to protect from moisture.",
        "Blister: Store in the original package in order to protect from moisture."
    ]
}'''
from json import JSONDecoder 
decoder = JSONDecoder()
# Fallback: ask Ollama to re-extract JSON via your llm
try:

    prompt = (
        "From this response extract the JSON data and return JSON data only, no extra text:\n\n"
        f"{text}"
    )

    # Correct call
    ollama_response = llm.invoke(prompt)
    print(ollama_response)
    # If the response is a string (likely), parse it
    new_content = ollama_response.content
    response_text = re.sub(r"^```json\s*|\s*```$", "", new_content, flags=re.IGNORECASE).strip()
    # Try parsing the cleaned-up output
    parsed, idx = decoder.raw_decode(response_text)
    if not isinstance(parsed, dict):
        raise ValueError("Ollama didn’t return a JSON object")
except Exception as e2:
    logging.error(f"[extract_fields_multi_step] Ollama fallback failed for batch: {e2!r}")

content='```json\n{\n    "shelf_life_time_period": "3 years",\n    "special_precautions_for_storage": [\n        "Do not store above 30° C.",\n        "Bottle: Keep the container tightly closed in order to protect from moisture.",\n        "Blister: Store in the original package in order to protect from moisture."\n    ]\n}\n```' additional_kwargs={} response_metadata={'model': 'phi4:latest', 'created_at': '2025-07-23T12:22:35.7661226Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1483787900, 'load_duration': 25268300, 'prompt_eval_count': 97, 'prompt_eval_duration': 2059100, 'eval_count': 75, 'eval_duration': 1455930900, 'model_name': 'phi4:latest'} id='run--cc087b12-2d5c-4238-b35c-c1dba96594a0-0' usage_metadata={'input_tokens': 97, 'output_tokens': 75, 'total_tokens': 172}


In [21]:
column_sums

A     6
B    15
dtype: int64

In [15]:
lang_filtered

List ID       Term ID  Term Rowid Language  \
4    100000073344  100000155939      104499       en   
9    100000073344  100000159361      105053       en   
10   100000073344  200000029586    16790002       en   
15   100000073344  200000043310    25930007       en   
19   100000073344  200000010387     3730001       en   
21   100000073344  200000010388     3750001       en   
25   100000073344  100000073410       68074       en   
44   100000073344  100000073411       68075       en   
61   100000073344  100000073412       68076       en   
85   100000073344  100000073413       68077       en   
109  100000073344  100000073414       68078       en   
133  100000073344  100000073415       68079       en   
160  100000073344  100000073416       68080       en   
181  100000073344  100000073417       68081       en   
205  100000073344  100000073418       68082       en   
229  100000073344  100000073419       68083       en   
252  100000073344  100000073420       68084       en   
275  100000073344  100000073421       68085       en   
299  100000073344  100000073422       68086       en   
323  100000073344  100000073423       68087       en   
347  100000073344  100000073424       68088       en   
371  100000073344  100000073425       68089       en   
395  100000073344  100000073426       68090       en   
419  100000073344  100000073427       68091       en   
444  100000073344  100000073428       68092       en   
454  100000073344  100000073429       68093       en   
465  100000073344  100000073430       68094       en   
476  100000073344  100000073431       68095       en   
486  100000073344  100000073432       68096       en   
496  100000073344  100000073433       68097       en   
506  100000073344  100000073434       68098       en   
513  100000073344  100000151831       99852       en   

                                             Term Name Term Name Modified On  \
4    Store at temperatures below -140°C in the vapo...   09/12/2013 09:00:17   
9                        Do not expose to extreme heat   10/11/2014 13:58:35   
10   Detailed special precautions for storage are p...   18/10/2022 11:03:02   
15                                        Do not shake   05/06/2024 09:08:42   
19                                   Protect from heat   09/08/2019 17:04:01   
21                                       Store upright   09/08/2019 17:09:01   
25                             Do not store above 25°C   21/02/2011 13:37:04   
44                             Do not store above 30°C   21/02/2011 13:37:31   
61                                    Store below 25°C   17/06/2010 13:25:19   
85                                    Store below 30°C   17/06/2010 13:26:26   
109                Store in a refrigerator (2°C – 8°C)   21/02/2011 13:38:15   
133       Store and transport refrigerated (2°C – 8°C)   21/02/2011 13:39:14   
160                                 Store in a freezer   17/06/2010 13:28:07   
181                         Store and transport frozen   21/02/2011 13:39:41   
205                       Do not refrigerate or freeze   21/02/2011 13:41:12   
229                                 Do not refrigerate   25/11/2015 17:21:26   
252                                      Do not freeze   03/12/2008 08:25:55   
275                      Store in the original package   03/12/2008 08:25:55   
299                  Keep the container tightly closed   21/02/2011 13:42:41   
323             Keep the container in the outer carton   21/02/2011 13:43:05   
347  This medicinal product does not require any sp...   21/02/2011 13:43:26   
371  This medicinal product does not require any sp...   21/02/2011 13:44:03   
395                     in order to protect from light   21/02/2011 13:44:23   
419                  in order to protect from moisture   21/02/2011 13:44:52   
444                                 Protect from frost   21/02/2011 13:45:17   
454                    Store in the original container   06/09/2010 20:10:50   
4

Change the language full name to short name

In [2]:
import pandas as pd
language_short = pd.read_csv("refrence files/language_short_name.csv",keep_default_na=False)
language_dic = {value.values[0]:value.values[1] for index,value in language_short.iterrows()}
shubstance = pd.read_csv("refrence files/sms-substances-list-20250613030058.csv",keep_default_na=False)
print("QO code ",shubstance )

QO code               #SMS_ID                        Substance_Name  Is_Preferred_Name  \
0       100000075987          HUMAN MEASLES IMMUNOGLOBULIN               True   
1       100000075987   IMMUNOGLOBULINE HUMAINE ROUGEOLEUSE              False   
2       100000075987  IMMUNOGLOBULINUM HUMANUM MORBILLICUM              False   
3       100000075987   Immunglobulin mot meslinger, humant              False   
4       100000075988        HUMAN PLASMA FOR FRACTIONATION               True   
...              ...                                   ...                ...   
216431  300000059106                                 LB-S8              False   
216432  300000059107                               DSPE-RB               True   
216433  300000059108                      DSPE-RB disodium               True   
216434  300000059109                             BP1.15205               True   
216435  300000059110                 Opadry 03B540176 pink               True   

         Language 

In [11]:
language_dic = {value.values[0].lower():value.values[1] for index,value in language_short.iterrows()}
language_dic

{'bulgarian': 'bg',
 'croatian': 'hr',
 'czech': 'cs',
 'danish': 'da',
 'dutch': 'nl',
 'english': 'en',
 'estonian': 'et',
 'finnish': 'fi',
 'french': 'fr',
 'german': 'de',
 'greek, modern (1453–)': 'el',
 'hungarian': 'hu',
 'icelandic': 'is',
 'irish': 'ga',
 'italian': 'it',
 'latin': 'la',
 'latvian': 'lv',
 'lithuanian': 'lt',
 'maltese': 'mt',
 'norwegian': 'no',
 'polish': 'pl',
 'portuguese': 'pt',
 'romanian': 'ro',
 'slovak': 'sk',
 'slovenian': 'sl',
 'spanish': 'es',
 'swedish': 'sv'}

In [6]:
for index, row in shubstance.iterrows():
    

Language    Bulgarian
Code               bg
Name: 0, dtype: object

In [ ]:
import util
import pandas as pd
route_of_administration_codes = pd.read_excel("refrence files/Route of Administration EMA Codes.xlsx",keep_default_na=False)
quantity_operator_codes = pd.read_excel("refrence files/Quantity operators EMA codes.xlsx",keep_default_na=False)
ROA_Code = util.get_codes(route_of_administration_codes,{"Term Name":"Oral use".lower()},'Identifier')
QO_Code = util.get_codes(quantity_operator_codes,{"Term Name":"equal to".lower()},'Identifier')
print("ROA code ",ROA_Code )
print("QO code ",QO_Code )


ROA code  100000073619
QO code  100000000049


In [ ]:
import codecs

escaped = "25\u00b0C"
actual = codecs.decode(escaped, 'unicode_escape')
print(actual)  # °C

25Â°C


In [76]:
import codecs
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("pdfs/Losec Control SmPC.pdf")
docs = loader.load()

for doc in docs:
    text = doc.page_content
    if "Do not store above" in doc.page_content:
            print("_"*100)
            print(doc)
    # If you ever spot literal 'Â' noise, repair it:
    if "Â°C" in text:
        text = text.encode("latin-1").decode("utf-8")

    # Now text contains the true '°' character
    doc.page_content = text

# proceed with splitting, embedding, etc.


____________________________________________________________________________________________________
page_content='BCH23004 9/9  
 
 
6.3 Shelf life 
 
3 years. 
 
6.4 Special precautions for storage 
 
Do not store above 25°C. 
Store in the original package in order to protect from moisture. 
 
6.5 Nature and contents of container 
 
Polyamide/Aluminium/PVC blister containing 7 or 14 tablets. 
 
Not all pack sizes may be marketed. 
 
6.6 Special precautions for disposal and other handling 
 
No special requirements. 
 
7. MARKETING AUTHORISATION HOLDER 
 
Bayer Limited 
1st Floor 
The Grange Offices 
The Grange 
Brewery Road 
Stillorgan 
Co. Dublin 
A94 H2K7 
Ireland 
 
8. MARKETING AUTHORISATION NUMBER(S) 
 
PA1410/066/001 
 
9. DATE OF FIRST AUTHORISATION/RENEWAL OF THE AUTHORISATION 
 
Date of first authorisation: 21st December 2011 
Date of latest renewal: 17th February 2016 
 
 
10. DATE OF REVISION OF THE TEXT 
 
March 2023' metadata={'producer': 'Adobe PDF Library 22.1.174', 'c

In [64]:
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.base import Embeddings
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import FAISS

# Configuration
PDF_PATH = "pdfs/Losec Control SmPC.pdf"
INDEX_PATH = "faiss_index"


# 1. Load PDF
# Option A: Using PyPDFLoader (optimized for PDFs)
loader = PyPDFLoader(PDF_PATH)
# Option B: Using UnstructuredPDFLoader (more general)
# loader = UnstructuredPDFLoader(PDF_PATH)

# 2. Split text into chunks
splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
docs = loader.load()
texts = splitter.split_documents(docs)

# 3. Build or load FAISS index
def build_index(texts, emb, index_path=INDEX_PATH):
    if os.path.exists(index_path):
        return FAISS.load_local(index_path, emb)
    vectorstore = FAISS.from_documents(texts, emb)
    vectorstore.save_local(index_path)
    return vectorstore

vectorstore = build_index(texts, embeddings)

# 4. Create a retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",  # or "mmr" for Maximal Marginal Relevance
    search_kwargs={"k": 5},
)

# 5. Simple query function
def retrieve(query: str):
    """
    Retrieve top-k relevant chunks for the given query.
    Returns a list of Document objects with .page_content and .metadata.
    """
    results = retriever.get_relevant_documents(query)
    return results

# Example usage
if __name__ == "__main__":
    q = "List of Excipient in '6. PHARMACEUTICAL PARTICULARS, 6.1 List of excipients'"
    hits = retrieve(q)
    for i, doc in enumerate(hits, 1):
        print(f"--- Chunk {i} (score metadata may be in doc.metadata) ---")
        print(doc.page_content)
        print()


--- Chunk 1 (score metadata may be in doc.metadata) ---
BCH23004 8/9  
 
Excretion 
The plasma elimination half -life of omeprazole is usually shorter than one hour both after single and  repeated 
oral once-daily dosing. Omeprazole is completely eliminated from plasma betwee n doses with no tendency for 
accumulation during o nce-daily administration. Almost 80% of an oral d ose of omeprazole is  excreted as 
metabolites in the urine, the remainder in the faeces, primarily originating from bile secretion. 
 
The AUC of omeprazole increases with repeated administration. This increase i s dose-dependent and results in 
a non-linear dose-AUC relationship after repeated administration. This time- and dose- dependency is due to a 
decrease of first pass metabolism and systemic clearance proba bly caused by an inhibition of the  CYP2C19 
enzyme by omeprazole and/or its metabolites (e.g. the su lphone). No metabolite has been found  to have an y 
effect on gastric acid secretion. 
 
Special 

In [15]:
import pandas as pd
instructions = pd.read_excel("refrence files/IDMP_Requirement_instructions.xlsx",sheet_name="Bot_Instructions",keep_default_na=False)
print(instructions)

           Output Section                     Field Name  \
0      Country / Language                        Country   
1      Country / Language                       Language   
2  Medicinal Product Name                      Full Name   
3  Medicinal Product Name             Invented Name Part   
4  Medicinal Product Name                  Strength Part   
5  Medicinal Product Name  Pharmaceutical Dose Form Part   

                              Section  \
0   7. Marketing authorisation holder   
1                                       
2   1. Name of the medicinal product    
3                                       
4   1. Name of the medicinal product    
5   1. Name of the medicinal product    

                                         Description  \
0  Extract full name of the Country or region for...   
1                  Detect Language from the pdf text   
2            Extract the whole name from the section   
3                                                      
4          

In [24]:
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Console handler
console_handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

# File handler
file_handler = logging.FileHandler('application.log')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
import json
import os
import re
from typing import List, Dict, Optional
from langchain.schema import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_community.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
import pandas as pd
instructions = pd.read_excel("refrence files/IDMP_Requirement_instructions.xlsx",sheet_name="Bot_Instructions",keep_default_na=False)
from dotenv import load_dotenv
load_dotenv()
# === Configuration ===
BASE_URL = os.getenv("BASE_URL")
EMBED_MODEL = os.getenv("EMBED_MODEL")
MODEL = os.getenv("MODEL")
logger.info("Loading Models")
# LLM initialization 
llm = ChatOllama(
    model=MODEL,
    temperature=0,
    base_url=BASE_URL,
)

# Embeddings initialization
embeddings = OllamaEmbeddings(
    model=EMBED_MODEL,
    base_url=BASE_URL,
)

def clean_page_text(text: str) -> str:
    lines = text.splitlines()
    cleaned = []
    for line in lines:
        # Skip header/footer patterns; adjust as needed
        if line.strip().startswith("http"):
            continue
        if re.match(r"\d+/\d+/\d+,\s*\d+:\d+\s*(AM|PM)", line.strip()):
            continue
        cleaned.append(line)
    return "\n".join(cleaned)

def build_vectorstore(
    pdf_path: str,
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
) -> FAISS:
    loader = PyPDFLoader(pdf_path)
    pages = loader.load_and_split()
    # Clean each page
    cleaned_pages = [clean_page_text(p.page_content) for p in pages]
    all_text = "\n\n".join(cleaned_pages)
    combined_doc = Document(page_content=all_text, metadata={})
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    docs = splitter.split_documents([combined_doc])
    logger.info(f"[build_vectorstore_combined] Created {len(docs)} chunks after combining pages.")
    vectorstore = FAISS.from_documents(docs, embeddings)
    return vectorstore


def retrieve_chunks(
    vectorstore: FAISS,
    query: str,
    k: int = 2
) -> List:
    """
    Retrieve top-k most relevant document chunks for the query.
    Prints the retrieved chunks for debugging.
    Returns the list of Document objects.
    """
    retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k": k})
    docs = retriever.get_relevant_documents(query)
    logger.info(f"[retrieve_chunks] Retrieved {len(docs)} chunks for query: {query!r}")
    for i, doc in enumerate(docs, start=1):
        print(f"--- Chunk {i} preview ---\n{doc.page_content}...\n")
    with open("debug_chunk.txt", "w", encoding="utf-8") as f:
        for i, doc in enumerate(docs, 1):
            f.write(f"--- Chunk {i} ---\n")
            f.write(doc.page_content + "\n\n")
    return docs


def build_llm_chain(llm) -> LLMChain:
    """
    Build an LLMChain with fixed system and human prompts.
    The system prompt instructs the model to output valid JSON only.
    The human prompt inserts the retrieved context and question.
    """
    system_prompt = (
        "You are a data extraction assistant. "
        "Output must be valid JSON ONLY."
    )
    human_template = (
        "Given the following extracted text from drug detail PDFs, answer the request below and "
        "return EXACTLY a JSON object with the keys specified (no extra keys).\n"
        "Return empty string if you cannot find values.\n\n"
        "Contex:\n\n{context}\n\n"
        "Request: {question}"
    )

    # Create a ChatPromptTemplate with system and user roles
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("user", human_template),
    ])
    # Build and return the LLMChain
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain


def format_retrieval_query_for_batch(
    batch: List[Dict[str, Optional[str]]]
) -> str:
    """
    Build a concise retrieval query from the batch.
    E.g., include field names and section hints to focus similarity search.
    """
    parts = []
    for info in batch:
        name = info.get("name")
        section = info.get("section", "").strip() if info.get("section") else ""
        if section:
            parts.append(f"{name} in section '{section}'")
        else:
            parts.append(name)
    # Join with commas; you could also add verbs like "find info about ..."
    query = " , ".join(parts)
    with open("retriever_prompt.txt", "w", encoding="utf-8") as f:
        for doc in parts:
            f.write(doc+ "\n\n")
    # Optionally prepend something like "Find text about ..."
    return f"{query}"


def format_question_for_batch(
    batch: List[Dict[str, Optional[str]]]
) -> str:
    """
    Given a batch of field-info dicts, build a question string that includes:
      - field name
      - description
      - optional section hint
    Returns a bullet-list style question for the LLM.
    """
    lines = ["Extract the following fields as JSON:"]
    for info in batch:
        name = info.get("name").strip().replace(" ","_").lower()
        desc = info.get("description", "").strip()
        section = info.get("section", "").strip() if info.get("section") else None
        bullet = f"- {name}"
        details = []
        if desc:
            details.append(f"description: '{desc}'")
        if section:
            details.append(f"section hint: '{section}'")
        if details:
            bullet += " (" + "; ".join(details) + ")"
        lines.append(bullet)
    with open("rag_prompt.txt", "w", encoding="utf-8") as f:
        for doc in lines:
            f.write(doc+ "\n\n")
    return "\n".join(lines)


def extract_fields_multi_step(
    vectorstore: FAISS,
    batches: List[List[Dict[str, Optional[str]]]],
    llm_chain: LLMChain,
    k: int = 5
) -> Dict[str, str]:
    """
    For each batch of field-info dicts:
      - Build separate retrieval query
      - Retrieve relevant chunks & print for debugging
      - Combine retrieved chunks as context
      - Build full LLM question
      - Invoke LLM chain to get JSON response
      - Parse and merge into final_result
    """
    final_result: Dict[str, str] = {}
    for out_section,batch in batches.items():
        # print(batch)
        # 1. Build retrieval query
        retrieval_query = format_retrieval_query_for_batch(batch)
        # 2. Retrieve relevant chunks
        # print("query to retriever \n", retrieval_query)
        docs = retrieve_chunks(vectorstore, retrieval_query, k=k)
        # 3. Combine retrieved content as context string
        context = "\n\n".join([doc.page_content for doc in docs])

        # 4. Build full LLM question
        question = format_question_for_batch(batch)

        # 5. Invoke the LLM chain
        try:
            ai_response = llm_chain.invoke({"context": context, "question": question})
            print("response keys = ", ai_response.keys())
            print("response = ", ai_response)
        except Exception as e:
            print(f"[extract_fields_multi_step] LLM chain invocation failed: {e}")
            continue
        response_text = ai_response["text"].strip()
        response_text = response_text.replace("`","")
        response_text = response_text.replace("json","")
        # 6. Parse JSON output
        try:
            parsed = json.loads(response_text)
            if not isinstance(parsed, dict):
                raise ValueError("Parsed JSON is not an object")
        except Exception as e:
            logging.error(f"[extract_fields_multi_step] Warning: failed to parse JSON for batch {batch}: {e!r}")
            logging.debug(f"Response was: {response_text!r}")
            continue

        # 7. Merge into final_result; later batches override earlier for same key
        for key, val in parsed.items():
            final_result[key] = val

    return final_result

2025-06-26 17:05:31,386 - INFO - Loading Models
2025-06-26 17:05:31,386 - INFO - Loading Models
2025-06-26 17:05:31,386 - INFO - Loading Models


In [157]:
import pandas as pd
import util
instructions = pd.read_excel("refrence files/IDMP_Requirement_instructions.xlsx",sheet_name="Bot_Instructions",keep_default_na=False)
def get_batches(instructions):
    """Create a dictionary of batches for sections from the bot instructions file"""
    batches = {}
    duplicate_batches = {}
    current_sec = None
    original_fields = []
    duplicate_fields = []

    for _, row in instructions.iterrows():
        out_sec = row['Output Section']

        # on section change, flush previous data if not empty
        if out_sec != current_sec:
            if current_sec is not None:
                if original_fields:
                    batches[current_sec] = original_fields
                if duplicate_fields:
                    duplicate_batches[current_sec] = duplicate_fields
            # reset lists for new section
            original_fields = []
            duplicate_fields = []
            current_sec = out_sec

        # build field dicts
        if not row['Existing']:
            fields = {
                'section': row['Section'],
                'name': row['Field Name'],
                'code': row['Code'],
                'keywords': row['Keywords'],
                'description': row['Description']
            }
            original_fields.append(fields)
        else:
            dup = {
                'section': row['Section'],
                'name': row['Field Name'],
                'Existing Section': row['Existing Section'],
                'code': row['Code'],
                'Existing Label': row['Existing Label']
            }
            duplicate_fields.append(dup)

    # flush last section if not empty
    if current_sec is not None:
        if original_fields:
            batches[current_sec] = original_fields
        if duplicate_fields:
            duplicate_batches[current_sec] = duplicate_fields

    return batches, duplicate_batches


In [159]:
batches, dublicate_batches = get_batches(instructions)
dublicate_batches

{'Marketing Authorisation': [{'section': '7. MARKETING AUTHORISATION HOLDER',
   'name': 'Country',
   'Existing Section': 'Country / Language',
   'code': True,
   'Existing Label': 'Country'}],
 'Medicinal Product': [{'section': '',
   'name': 'Authorised Pharmaceutical Form',
   'Existing Section': 'Medicinal Product Name',
   'code': True,
   'Existing Label': 'Pharmaceutical Dose Form Part'},
  {'section': '',
   'name': 'Language',
   'Existing Section': 'Country / Language',
   'code': True,
   'Existing Label': 'Language'}],
 'Packaged Medicinal Product': [{'section': '',
   'name': 'Language',
   'Existing Section': 'Country / Language',
   'code': True,
   'Existing Label': 'Language'}],
 'Package Item (Container) (2)': [{'section': '',
   'name': 'Package Item (Container) Quantity',
   'Existing Section': 'Package Item (Container) (1)',
   'code': False,
   'Existing Label': 'Package Item (Container) Quantity'}],
 'Pharmaceutical Product': [{'section': '',
   'name': 'Admini

In [11]:
batches.keys()

dict_keys(['Country / Language', 'Medicinal Product Name', 'Shelf Life', 'Marketing Authorisation', 'Route of Administration', 'List of Excipients', 'Active Ingredient', 'Packaged Medicinal Product', 'Package Item (Container) (2)'])

In [ ]:
import pandas as pd
codes = pd.read_csv("refrence files/DataSets/Country-codes.csv",keep_default_na=False)

In [ ]:
def get_codes(source_file, filters: dict,output):
    condition = True
    for key, value in filters.items():
        condition &= source_file[key].str.lower() == str(value).lower()
    filtered = source_file[condition]

    if len(filtered) > 0:
        print(filtered[output].values[0])
    print(len(source_file), len(filtered))
get_codes(codes,{"Language":"en",'Short Name':'ireland'},'Term ID')

0       False
1       False
2       False
3       False
4       False
        ...  
3770    False
3771    False
3772    False
3773    False
3774    False
Name: Language, Length: 3775, dtype: bool
100000000427
3775 1


In [26]:
batches = get_batches()
batches

{'Country / Language': [{'section': '7. MARKETING AUTHORISATION HOLDER',
   'name': 'Country',
   'description': 'Extract full name of the Country or region for which this document applies'},
  {'section': '',
   'name': 'Language',
   'description': 'Detect Language from the pdf text'}],
 'Medicinal Product Name': [{'section': '1. NAME OF THE MEDICINAL PRODUCT',
   'name': 'Full Name',
   'description': 'Extract the whole name from the section'},
  {'section': '', 'name': 'Invented Name Part', 'description': ''},
  {'section': '1. NAME OF THE MEDICINAL PRODUCT',
   'name': 'Strength Part',
   'description': 'Strength per unit, e.g., \'500 mg\'"'},
  {'section': '3. PHARMACEUTICAL FORM',
   'name': 'Pharmaceutical Dose Form Part',
   'description': 'Pharmaceutical Dose Form Part, e.g., tablet, capsule, injection'}],
 'Shelf Life': [{'section': '6.3 Shelf life',
   'name': 'Shelf Life Time Period',
   'description': 'Shelf life of the drug in years'},
  {'section': '6.4 Special precauti

In [27]:
def main(PDF_PATH):
    # === User parameters ===
    # PDF path configured at top (PDF_PATH)
    # Batches: list of lists of field-info dicts
    batches = get_batches()
    # 1. Build vectorstore from PDF
    logger.info("Building FAISS vectorstore from PDF...")
    vectorstore = build_vectorstore(PDF_PATH, chunk_size=int(os.getenv("CHUNK_SIZE",1000)), chunk_overlap=int(os.getenv("CHUNK_OVERLAP", 200)))

    # 2. Build LLMChain with customizable system prompt
    llm_chain = build_llm_chain(llm)

    # 3. Run multi-step extraction
    logger.info("Starting multi-step extraction...")
    result = extract_fields_multi_step(
        vectorstore=vectorstore,
        batches=batches,
        llm_chain=llm_chain,
        k=int(os.getenv("TOP_K", 2))  # e.g., retrieve top-2 chunks per query
    )
    return json.dumps(result, indent=2)

In [28]:
PDF_PATH = "pdfs/Losec Capsules 20mg (SmPC) -Chapter8-Example.pdf" 
result = main(PDF_PATH)
print("Final extracted result:")
print(result)

2025-06-26 17:05:40,005 - INFO - Building FAISS vectorstore from PDF...
2025-06-26 17:05:40,005 - INFO - Building FAISS vectorstore from PDF...
2025-06-26 17:05:40,005 - INFO - Building FAISS vectorstore from PDF...
2025-06-26 17:05:40,863 - INFO - [build_vectorstore_combined] Created 51 chunks after combining pages.
2025-06-26 17:05:40,863 - INFO - [build_vectorstore_combined] Created 51 chunks after combining pages.
2025-06-26 17:05:40,863 - INFO - [build_vectorstore_combined] Created 51 chunks after combining pages.
2025-06-26 17:05:42,348 - INFO - Starting multi-step extraction...
2025-06-26 17:05:42,348 - INFO - Starting multi-step extraction...
2025-06-26 17:05:42,348 - INFO - Starting multi-step extraction...
2025-06-26 17:05:42,423 - INFO - [retrieve_chunks] Retrieved 2 chunks for query: "Country in section '7. MARKETING AUTHORISATION HOLDER' , Language"
2025-06-26 17:05:42,423 - INFO - [retrieve_chunks] Retrieved 2 chunks for query: "Country in section '7. MARKETING AUTHORISAT

--- Chunk 1 preview ---
6.6 Special precautions for disposal and other handling
No special requirements.
7. Marketing authorisation holder
Neon Healthcare Ltd.
8 The Chase, John Tate Road,
Hertford,
SG13 7NN
United Kingdom
8. Marketing authorisation number(s)
PL 45043/0101
9. Date of first authorisation/renewal of the authorisation
28/02/2011
10. Date of revision of the text
30.08.2023
Company Contact Details
Neon Healthcare Ltd...

--- Chunk 2 preview ---
+44 (0)1992 926 330
Stock Availability
+44 (0)1992 926 330...



2025-06-26 17:05:44,825 - INFO - [retrieve_chunks] Retrieved 2 chunks for query: "Full Name in section '1. NAME OF THE MEDICINAL PRODUCT' , Invented Name Part , Strength Part in section '1. NAME OF THE MEDICINAL PRODUCT' , Pharmaceutical Dose Form Part in section '3. PHARMACEUTICAL FORM'"
2025-06-26 17:05:44,825 - INFO - [retrieve_chunks] Retrieved 2 chunks for query: "Full Name in section '1. NAME OF THE MEDICINAL PRODUCT' , Invented Name Part , Strength Part in section '1. NAME OF THE MEDICINAL PRODUCT' , Pharmaceutical Dose Form Part in section '3. PHARMACEUTICAL FORM'"
2025-06-26 17:05:44,825 - INFO - [retrieve_chunks] Retrieved 2 chunks for query: "Full Name in section '1. NAME OF THE MEDICINAL PRODUCT' , Invented Name Part , Strength Part in section '1. NAME OF THE MEDICINAL PRODUCT' , Pharmaceutical Dose Form Part in section '3. PHARMACEUTICAL FORM'"


response keys =  dict_keys(['context', 'question', 'text'])
response =  {'context': '6.6 Special precautions for disposal and other handling\nNo special requirements.\n7. Marketing authorisation holder\nNeon Healthcare Ltd.\n8 The Chase, John Tate Road,\nHertford,\nSG13 7NN\nUnited Kingdom\n8. Marketing authorisation number(s)\nPL 45043/0101\n9. Date of first authorisation/renewal of the authorisation\n28/02/2011\n10. Date of revision of the text\n30.08.2023\nCompany Contact Details\nNeon Healthcare Ltd\n\n+44 (0)1992 926 330\nStock Availability\n+44 (0)1992 926 330', 'question': "Extract the following fields as JSON:\n- country (description: 'Extract full name of the Country or region for which this document applies'; section hint: '7. MARKETING AUTHORISATION HOLDER')\n- language (description: 'Detect Language from the pdf text')", 'text': '```json\n{\n  "country": "United Kingdom",\n  "language": "English"\n}\n```'}
--- Chunk 1 preview ---
made after treatment with H2-receptor antago

2025-06-26 17:05:46,580 - INFO - [retrieve_chunks] Retrieved 2 chunks for query: "Shelf Life Time Period in section '6.3 Shelf life' , Special Precautions for Storage  in section '6.4 Special precautions for storage'"
2025-06-26 17:05:46,580 - INFO - [retrieve_chunks] Retrieved 2 chunks for query: "Shelf Life Time Period in section '6.3 Shelf life' , Special Precautions for Storage  in section '6.4 Special precautions for storage'"
2025-06-26 17:05:46,580 - INFO - [retrieve_chunks] Retrieved 2 chunks for query: "Shelf Life Time Period in section '6.3 Shelf life' , Special Precautions for Storage  in section '6.4 Special precautions for storage'"


response keys =  dict_keys(['context', 'question', 'text'])
response =  {'context': 'made after treatment with H2-receptor antagonists, proton pump inhibitors and after partial fundectomy. Thus, these\nchanges are not from a direct effect of any individual active substance.\n6. Pharmaceutical particulars\n6.1 List of excipients\nContent\nDisodium phosphate dihydrate,\nLow-substituted hydroxypropylcellulose,\nHypromellose,\nLactose anhydrous,\nMagnesium stearate,\nMannitol,\nMethacrylic acid – ethyl acrylate copolymer (1:1) dispersion 30 per cent,\nCellulose microcrystalline,\nMacrogol (polyethylene glycol 400),\nSodium laurilsulfate,\nCapsule shell\nIron oxide E172,\nTitanium dioxide E171,\nGelatine,\nMagnesium stearate,\nSodium laurilsulfate,\nPrinting ink (containing shellac, ammonia, potassium hydroxide and black iron oxide E172),\nSilica colloidal anhydrous,\nParaffin liquid\n\nLosec Capsules 20mg\nSummary of Product Characteristics Updated 06-Sep-2023 | Neon Healthcare Ltd\n1. Nam

2025-06-26 17:05:48,593 - INFO - [retrieve_chunks] Retrieved 2 chunks for query: "Marketing Authorisation Number in section '8. MARKETING AUTHORISATION NUMBER(S)' , Country in section '7. MARKETING AUTHORISATION HOLDER' , Authorisation Status Date in section '9. DATE OF FIRST AUTHORISATION/RENEWAL OF THE AUTHORISATION' , Date of First Authorisation in section '9. DATE OF FIRST AUTHORISATION/RENEWAL OF THE AUTHORISATIO'"
2025-06-26 17:05:48,593 - INFO - [retrieve_chunks] Retrieved 2 chunks for query: "Marketing Authorisation Number in section '8. MARKETING AUTHORISATION NUMBER(S)' , Country in section '7. MARKETING AUTHORISATION HOLDER' , Authorisation Status Date in section '9. DATE OF FIRST AUTHORISATION/RENEWAL OF THE AUTHORISATION' , Date of First Authorisation in section '9. DATE OF FIRST AUTHORISATION/RENEWAL OF THE AUTHORISATIO'"
2025-06-26 17:05:48,593 - INFO - [retrieve_chunks] Retrieved 2 chunks for query: "Marketing Authorisation Number in section '8. MARKETING AUTHORISATION 

response keys =  dict_keys(['context', 'question', 'text'])
response =  {'context': 'Address\n8 The Chase, John Tate Road, Foxholes Business\nPark, Hertford, Hertfordshire, SG13 7NN, UK\nTelephone\n+44 (0)1992 926 330\nMedical Information e-mail\nmedinfo@neonhealthcare.com\nStock Availability\noffice@neonhealthcare.com\nWWW\nMedical Information Direct Line\n+44 (0)1992 926 330\nCustomer Care direct line\n6.2 Incompatibilities\nNot applicable.\n6.3 Shelf life\n3 years.\n6.4 Special precautions for storage\nDo not store above 30° C.\nBottle: Keep the container tightly closed in order to protect from moisture.\nBlister: Store in the original package in order to protect from moisture.\n6.5 Nature and contents of container\nHDPE bottle: with a tight fitting polypropylene screw-cap equipped with a desiccant capsule.\n20 mg: 7, 14, 28, 30 capsules.\nAluminium blister.\n20 mg: 14, 28 capsules.\nNot all pack sizes may be marketed.\n6.6 Special precautions for disposal and other handling\nNo spe

2025-06-26 17:05:50,187 - INFO - [retrieve_chunks] Retrieved 2 chunks for query: 'Route of Administration'
2025-06-26 17:05:50,187 - INFO - [retrieve_chunks] Retrieved 2 chunks for query: 'Route of Administration'
2025-06-26 17:05:50,187 - INFO - [retrieve_chunks] Retrieved 2 chunks for query: 'Route of Administration'


response keys =  dict_keys(['context', 'question', 'text'])
response =  {'context': '6.6 Special precautions for disposal and other handling\nNo special requirements.\n7. Marketing authorisation holder\nNeon Healthcare Ltd.\n8 The Chase, John Tate Road,\nHertford,\nSG13 7NN\nUnited Kingdom\n8. Marketing authorisation number(s)\nPL 45043/0101\n9. Date of first authorisation/renewal of the authorisation\n28/02/2011\n10. Date of revision of the text\n30.08.2023\nCompany Contact Details\nNeon Healthcare Ltd\n\n+44 (0)1992 926 330\nStock Availability\n+44 (0)1992 926 330', 'question': "Extract the following fields as JSON:\n- marketing_authorisation_number (section hint: '8. MARKETING AUTHORISATION NUMBER(S)')\n- country (description: 'Extract full name of the Country or region for which this document applies'; section hint: '7. MARKETING AUTHORISATION HOLDER')\n- authorisation_status_date (description: 'Extract the date in YYYY-MM-DD format from last date of renewal'; section hint: '9. DAT

In [5]:
material_codes

List ID       Term ID  Term Rowid Language  \
0    2.000000e+11  2.000000e+11    11430002       en   
1    2.000000e+11  2.000000e+11    11430002       no   
2    2.000000e+11  2.000000e+11    11430002       es   
3    2.000000e+11  2.000000e+11    11430003       en   
4    2.000000e+11  2.000000e+11    11430003       no   
..            ...           ...         ...      ...   
256  2.000000e+11  2.000000e+11     6530001       es   
257  2.000000e+11  2.000000e+11     6530001       en   
258  2.000000e+11  2.000000e+11     7750001       no   
259  2.000000e+11  2.000000e+11     7750001       es   
260  2.000000e+11  2.000000e+11     7750001       en   

                       Term Name Term Name Modified On Term Name Modified By  \
0          Expanded Polyethylene      21-09-2021 10:30                         
1          Ekspandert polyetylen      01-06-2022 11:02                         
2          Polietileno expandido      22-02-2023 12:33                         
3          Cyclic Olefin Polymer      21-09-2021 13:01                         
4          Syklisk olefinpolymer      01-06-2022 11:02                         
..                           ...                   ...                   ...   
256  Polipropileno sindiotáctico      22-02-2023 12:33                         
257   Syndiotactic Polypropylene      22-06-2020 10:09                         
258          Polyetylenkopolymer      01-06-2022 11:02                         
259    Copolímero de polietileno      22-02-2023 12:33                         
260       PolyEthylene copolymer      12-11-2020 19:51                         

    Term Name Translation Status Term Name Comments  Term Name Rowid  ...  \
0                        CURRENT                            12730001  ...   
1                        CURRENT                            17410037  ...   
2                        CURRENT                            21950010  ...   
3                        CURRENT                            12730003  ...   
4                        CURRENT                            17430032  ...   
..                           ...                ...              ...  ...   
256                      CURRENT                            21990010  ...   
257                      CURRENT                             7690001  ...   
258                      CURRENT                            17430031  ...   
259                      CURRENT                            21930010  ...   
260                      CURRENT                             9010001  ...   

    Term Attribute 7 Rowid Attribute 8 Attribute Value 8  \
0                                                          
1                                                          
2                                                          
3                                                          
4                                                          
..                     ...         ...               ...   
256                                                        
257                                                        
258                                                        
259                                                        
260                                                        

    Term Attribute 8 Rowid Attribute 9 Attribute Value 9  \
0                                                          
1                                                          
2                                                          
3                                                          
4                                                          
..                     ...         ...               ...   
256                                                        
257                                                        
258                                                        
259                                                        
260                                                        

    Term 

In [5]:
import pandas as pd
import util
val = ['Polyamide', 'Aluminium', 'PVC']
material_codes = pd.read_csv("refrence files/materials_code.csv",keep_default_na=False)
material_with_code = []
for material in val:
    code = util.get_codes(material_codes,{"Language":"en","Term Name":material.lower()},'Term ID')
    print(f"code = {code}")
    material_with_code.append(f'{material} -  ({code})')

200000000000.0
code = 200000000000.0
200000000000.0
code = 200000000000.0
code = 


In [4]:
material_with_code


['Polyamide -  (200000000000.0)', 'Aluminium -  (200000000000.0)', 'PVC -  ()']

In [42]:
from dotenv import load_dotenv
import os
from langchain_ollama import OllamaEmbeddings, ChatOllama

load_dotenv()
# === Configuration ===
BASE_URL = os.getenv("BASE_URL")
EMBED_MODEL = os.getenv("EMBED_MODEL")
MODEL = os.getenv("MODEL")
# LLM initialization 
llm = ChatOllama(
    model=MODEL,
    temperature=0,
    base_url=BASE_URL,
)

# Embeddings initialization
embeddings = OllamaEmbeddings(
    model=EMBED_MODEL,
    base_url=BASE_URL,
)

In [ ]:
from typing import List, Optional, Tuple
from pathlib import Path

from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import FAISS

def process_and_index_pdf(
    pdf_path: str,
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
    loader_type: str = "pypdf",
    embeddings_model: str = "mxbai-embed-large:latest",
    embeddings_base_url: str = "http://192.168.0.15:11434",
) -> Tuple[FAISS, List[str]]:
    """
    Reads a PDF, splits it into overlapping chunks, embeds them, and indexes in FAISS.
    
    Args:
        pdf_path (str): Path to the PDF file.
        chunk_size (int): Max characters per chunk.
        chunk_overlap (int): Overlap between chunks.
        loader_type (str): Which loader to use: "pypdf" or "unstructured".
        embeddings_model (str): Ollama model name for embeddings.
        embeddings_base_url (str): Ollama server URL.
    
    Returns:
        faiss_index (FAISS): FAISS index ready for similarity search.
        texts (List[str]): The raw text chunks.
    """
    # 1. LOAD THE DOCUMENT
    pdf_file = Path(pdf_path)
    if not pdf_file.exists():
        raise FileNotFoundError(f"File not found: {pdf_path}")
    
    if loader_type == "pypdf":
        loader = PyPDFLoader(str(pdf_file))
    elif loader_type == "unstructured":
        loader = UnstructuredPDFLoader(str(pdf_file))
    else:
        raise ValueError(f"Unsupported loader_type: {loader_type}")
    
    docs = loader.load()  # returns List[Document]
    
    # 2. SPLIT INTO CHUNKS
    splitter = CharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        keep_separator=True
    )
    texts = splitter.split_documents(docs)
    
   
    
    # 4. INDEX IN FAISS
    faiss_index = FAISS.from_documents(texts, embeddings)
    
    return faiss_index, [t.page_content for t in texts]


# === Example usage ===
if __name__ == "__main__":
    INDEX, chunks = process_and_index_pdf(
        pdf_path="pdfs/Losec Control SmPC.pdf",
        chunk_size=1000,
        chunk_overlap=200,
        loader_type="pypdf",
        embeddings_model="EMBED_MODEL",
        embeddings_base_url="BASE_URL"
    )
    # Now you can do:
    results = INDEX.similarity_search("Full Name in '1. NAME OF THE MEDICINAL PRODUCT' , Invented Name Part , Strength Part in '1. NAME OF THE MEDICINAL PRODUCT' , Pharmaceutical Dose Form Part in '1. NAME OF THE MEDICINAL PRODUCT", k=5)
    print(f"Indexed {len(chunks)} chunks.")
    print(results)


Indexed 9 chunks.
[Document(id='30ec9940-e07b-4d83-bf17-d056002fd443', metadata={'producer': 'Adobe PDF Library 22.1.174', 'creator': 'Acrobat PDFMaker 22 for Word', 'creationdate': '2023-04-03T12:55:16+01:00', 'author': 'AIFA', 'company': 'AIFA', 'keywords': '', 'msip_label_7f850223-87a8-40c3-9eb2-432606efca2a_actionid': 'd3625aaa-fc16-402f-869b-64af1a766544', 'msip_label_7f850223-87a8-40c3-9eb2-432606efca2a_contentbits': '0', 'msip_label_7f850223-87a8-40c3-9eb2-432606efca2a_enabled': 'true', 'msip_label_7f850223-87a8-40c3-9eb2-432606efca2a_method': 'Privileged', 'msip_label_7f850223-87a8-40c3-9eb2-432606efca2a_name': '7f850223-87a8-40c3-9eb2-432606efca2a', 'msip_label_7f850223-87a8-40c3-9eb2-432606efca2a_setdate': '2022-12-21T18:27:43Z', 'msip_label_7f850223-87a8-40c3-9eb2-432606efca2a_siteid': 'fcb2b37b-5da0-466b-9b83-0014b67a7c78', 'moddate': '2025-04-15T09:20:00+05:30', 'sourcemodified': 'D:20230315113802', 'subject': '', 'title': 'I', 'source': 'pdfs\\Losec Control SmPC.pdf', 'to

In [20]:
results

[Document(id='30ec9940-e07b-4d83-bf17-d056002fd443', metadata={'producer': 'Adobe PDF Library 22.1.174', 'creator': 'Acrobat PDFMaker 22 for Word', 'creationdate': '2023-04-03T12:55:16+01:00', 'author': 'AIFA', 'company': 'AIFA', 'keywords': '', 'msip_label_7f850223-87a8-40c3-9eb2-432606efca2a_actionid': 'd3625aaa-fc16-402f-869b-64af1a766544', 'msip_label_7f850223-87a8-40c3-9eb2-432606efca2a_contentbits': '0', 'msip_label_7f850223-87a8-40c3-9eb2-432606efca2a_enabled': 'true', 'msip_label_7f850223-87a8-40c3-9eb2-432606efca2a_method': 'Privileged', 'msip_label_7f850223-87a8-40c3-9eb2-432606efca2a_name': '7f850223-87a8-40c3-9eb2-432606efca2a', 'msip_label_7f850223-87a8-40c3-9eb2-432606efca2a_setdate': '2022-12-21T18:27:43Z', 'msip_label_7f850223-87a8-40c3-9eb2-432606efca2a_siteid': 'fcb2b37b-5da0-466b-9b83-0014b67a7c78', 'moddate': '2025-04-15T09:20:00+05:30', 'sourcemodified': 'D:20230315113802', 'subject': '', 'title': 'I', 'source': 'pdfs\\Losec Control SmPC.pdf', 'total_pages': 9, 'pa

In [51]:
from typing import List, Optional, Tuple
from pathlib import Path

from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.retrievers.bm25 import BM25Retriever


def build_vectorstore(
    pdf_path: str,
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
    loader_type: str = "pypdf",
    embeddings_model: str = "BM2",
    embeddings_base_url: str = "http://localhost:11434"
) -> Tuple[FAISS, List[str], BM25Retriever]:
    """
    Builds and returns vectorstore, text chunks, and BM25 retriever.
    """
    # Validate file exists
    pdf_file = Path(pdf_path)
    if not pdf_file.exists():
        raise FileNotFoundError(f"PDF not found: {pdf_path}")

    # Load document
    loader = {
        "pypdf": PyPDFLoader,
        "unstructured": UnstructuredPDFLoader
    }.get(loader_type)

    if not loader:
        raise ValueError(f"Unsupported loader_type: {loader_type}")
    docs = loader(str(pdf_file)).load()

    # Split into chunks
    splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, keep_separator=True)
    split_docs = splitter.split_documents(docs)

    # # Embeddings using Ollama BM2
    # embeddings = OllamaEmbeddings(model=embeddings_model, base_url=embeddings_base_url)

    # Create FAISS index
    faiss_index = FAISS.from_documents(split_docs, embeddings)

    # Create BM25 retriever
    bm25_retriever = BM25Retriever.from_documents(split_docs)
    bm25_retriever.k = 5

    return faiss_index, [doc.page_content for doc in split_docs], bm25_retriever


def retrieve_chunks(
    faiss_index: FAISS,
    texts: List[str],
    bm25_retriever: BM25Retriever,
    query: str,
    mode: str = "similarity",
    k: int = 5
) -> List[str]:
    """
    Retrieves matching chunks based on the mode.
    """
    if mode == "similarity":
        retriever = faiss_index.as_retriever(search_type="similarity", search_kwargs={"k": k})
    elif mode == "keyword":
        bm25_retriever.k = k
        retriever = bm25_retriever
    elif mode == "hybrid":
        sim_retriever = faiss_index.as_retriever(search_type="similarity", search_kwargs={"k": k})
        bm25_retriever.k = k

        class HybridRetriever:
            def __init__(self):
                self.sim = sim_retriever
                self.bm = bm25_retriever
                self.k = k

            def get_relevant_documents(self, query: str) -> List[str]:
                sim_docs = self.sim.get_relevant_documents(query)
                bm_docs = self.bm.get_relevant_documents(query)
                seen, combined = set(), []
                for doc in sim_docs + bm_docs:
                    if doc.page_content not in seen:
                        seen.add(doc.page_content)
                        combined.append(doc)
                    if len(combined) >= self.k:
                        break
                return combined

        retriever = HybridRetriever()
    else:
        raise ValueError("mode must be one of: similarity, keyword, hybrid")

    results = retriever.get_relevant_documents(query)
    return [doc.page_content if hasattr(doc, "page_content") else doc for doc in results]


In [53]:
faiss_index, texts, bm25 = build_vectorstore(
    pdf_path="pdfs/Losec Control SmPC.pdf",
    loader_type="pypdf",
    chunk_size=1000,
    chunk_overlap=200,
    embeddings_model="BM2",
    embeddings_base_url="http://192.168.0.15:11434"
)

results = retrieve_chunks(
    faiss_index=faiss_index,
    texts=texts,
    bm25_retriever=bm25,
    query="List of Excipient in '6.1 List of excipients'",
    mode="hybrid",
    k=2
)

for i, result in enumerate(results, 1):
    print(f"\n--- Result {i} ---\n{result}")



--- Result 1 ---
BCH23004 9/9  
 
 
6.3 Shelf life 
 
3 years. 
 
6.4 Special precautions for storage 
 
Do not store above 25°C. 
Store in the original package in order to protect from moisture. 
 
6.5 Nature and contents of container 
 
Polyamide/Aluminium/PVC blister containing 7 or 14 tablets. 
 
Not all pack sizes may be marketed. 
 
6.6 Special precautions for disposal and other handling 
 
No special requirements. 
 
7. MARKETING AUTHORISATION HOLDER 
 
Bayer Limited 
1st Floor 
The Grange Offices 
The Grange 
Brewery Road 
Stillorgan 
Co. Dublin 
A94 H2K7 
Ireland 
 
8. MARKETING AUTHORISATION NUMBER(S) 
 
PA1410/066/001 
 
9. DATE OF FIRST AUTHORISATION/RENEWAL OF THE AUTHORISATION 
 
Date of first authorisation: 21st December 2011 
Date of latest renewal: 17th February 2016 
 
 
10. DATE OF REVISION OF THE TEXT 
 
March 2023

--- Result 2 ---
BCH23004 1/9  
 
Summary of Product Characteristics 
 
1. NAME OF THE MEDICINAL PRODUCT 
 
Losec Control 20 mg gastro-resistant tablet

In [55]:
for i,doc in enumerate(texts):
    print("chunck ", i)
    print(doc)
    print("*"*100)

chunck  0
BCH23004 1/9  
 
Summary of Product Characteristics 
 
1. NAME OF THE MEDICINAL PRODUCT 
 
Losec Control 20 mg gastro-resistant tablets 
 
2. QUALITATIVE AND QUANTITATIVE COMPOSITION 
 
Each gastro-resistant tablet contains 20.6 mg omeprazole magnesium equivalent to 20 mg omeprazole. 
Excipients with known effect:  
Each gastro-resistant tablet contains 19–20 mg sucrose. 
 
For the full list of excipients, see section 6.1. 
 
3. PHARMACEUTICAL FORM 
 
Gastro-resistant tablet. 
Pink, oblong, biconvex, film -coated tablets engraved with 
on one side and 20 mg on the other side , 
containing enteric coated pellets. 
 
4. CLINICAL PARTICULARS 
 
4.1 Therapeutic indications 
 
Losec Control gastro-resistant tablets is indicated for the treatment of reflux symptoms (e.g. heartburn, acid  
regurgitation) in adults. 
 
4.2 Posology and method of administration 
 
Posology in adults 
The recommended dose is 20 mg once daily for 14 days. 
It might be necessary to take the tablets for 2

In [34]:
index = read_and_index_pdf(
    pdf_path="pdfs/Losec Control SmPC.pdf",
    loader="unstructured",
    chunk_size=1000,
    chunk_overlap=200,
    embed_model="OllamaEmbeddingModelName",
    base_url="http://localhost:11434",
    faiss_index_path="my_faiss_index"
)

# e.g. to query:
docs_and_scores = index.similarity_search_with_score("Full Name in '1. NAME OF THE MEDICINAL PRODUCT' , Invented Name Part , Strength Part in '1. NAME OF THE MEDICINAL PRODUCT' , Pharmaceutical Dose Form Part in '1. NAME OF THE MEDICINAL PRODUCT", k=5)


ImportError: cannot import name 'open_filename' from 'pdfminer.utils' (c:\Users\thamilarasan.p\OneDrive - Globalvalueweb\Desktop\IDMP Datamate\IDMP_Datamate\venv\Lib\site-packages\pdfminer\utils.py)

In [3]:
import re
from langchain.schema import Document

def clean_page_text(text: str) -> str:
    lines = text.splitlines()
    cleaned = []
    for line in lines:
        # Skip header/footer patterns; adjust as needed
        if line.strip().startswith("http"):
            continue
        if re.match(r"\d+/\d+/\d+,\s*\d+:\d+\s*(AM|PM)", line.strip()):
            continue
        cleaned.append(line)
    return "\n".join(cleaned)

def build_vectorstore(
    pdf_path: str,
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
) -> FAISS:
    loader = PyPDFLoader(pdf_path)
    pages = loader.load_and_split()
    # Clean each page
    cleaned_pages = [clean_page_text(p.page_content) for p in pages]
    all_text = "\n\n".join(cleaned_pages)
    combined_doc = Document(page_content=all_text, metadata={})
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    docs = splitter.split_documents([combined_doc])
    print(f"[build_vectorstore_combined] Created {len(docs)} chunks after combining pages.")
    vectorstore = FAISS.from_documents(docs, embeddings)
    return vectorstore


def retrieve_chunks(
    vectorstore: FAISS,
    query: str,
    k: int = 2
) -> List:
    """
    Retrieve top-k most relevant document chunks for the query.
    Prints the retrieved chunks for debugging.
    Returns the list of Document objects.
    """
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})
    docs = retriever.get_relevant_documents(query)
    print(f"[retrieve_chunks] Retrieved {len(docs)} chunks for query: {query!r}")
    for i, doc in enumerate(docs, start=1):
        print(f"--- Chunk {i} preview ---\n{doc.page_content}...\n")
    with open("debug_chunk.txt", "w", encoding="utf-8") as f:
        for i, doc in enumerate(docs, 1):
            f.write(f"--- Chunk {i} ---\n")
            f.write(doc.page_content + "\n\n")
    return docs


def build_llm_chain(llm) -> LLMChain:
    """
    Build an LLMChain with fixed system and human prompts.
    The system prompt instructs the model to output valid JSON only.
    The human prompt inserts the retrieved context and question.
    """
    system_prompt = (
        "You are a data extraction assistant. "
        "Output must be valid JSON ONLY."
    )
    human_template = (
        "Given the following extracted text from drug detail PDFs, answer the request below and "
        "return EXACTLY a JSON object with the keys specified (no extra keys).\n\n"
        "{context}\n\n"
        "Request: {question}"
    )

    # Create a ChatPromptTemplate with system and user roles
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("user", human_template),
    ])
    print(prompt)
    # Build and return the LLMChain
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain


def format_retrieval_query_for_batch(
    batch: List[Dict[str, Optional[str]]]
) -> str:
    """
    Build a concise retrieval query from the batch.
    E.g., include field names and section hints to focus similarity search.
    """
    parts = []
    for info in batch:
        name = info.get("name")
        section = info.get("section", "").strip() if info.get("section") else ""
        if section:
            parts.append(f"{name} in section '{section}'")
        else:
            parts.append(name)
    # Join with commas; you could also add verbs like "find info about ..."
    query = " , ".join(parts)
    # Optionally prepend something like "Find text about ..."
    return f"{query}"


def format_question_for_batch(
    batch: List[Dict[str, Optional[str]]]
) -> str:
    """
    Given a batch of field-info dicts, build a question string that includes:
      - field name
      - description
      - optional section hint
    Returns a bullet-list style question for the LLM.
    """
    lines = ["Extract the following fields as JSON:"]
    for info in batch:
        name = info.get("name").strip().replace(" ","_").lower()
        desc = info.get("description", "").strip()
        section = info.get("section", "").strip() if info.get("section") else None
        bullet = f"- {name}"
        details = []
        if desc:
            details.append(f"description: '{desc}'")
        if section:
            details.append(f"section hint: '{section}'")
        if details:
            bullet += " (" + "; ".join(details) + ")"
        lines.append(bullet)
    return "\n".join(lines)


def extract_fields_multi_step(
    vectorstore: FAISS,
    batches: List[List[Dict[str, Optional[str]]]],
    llm_chain: LLMChain,
    k: int = 5
) -> Dict[str, str]:
    """
    For each batch of field-info dicts:
      - Build separate retrieval query
      - Retrieve relevant chunks & print for debugging
      - Combine retrieved chunks as context
      - Build full LLM question
      - Invoke LLM chain to get JSON response
      - Parse and merge into final_result
    """
    final_result: Dict[str, str] = {}
    for batch in batches:
        # 1. Build retrieval query
        retrieval_query = format_retrieval_query_for_batch(batch)
        # 2. Retrieve relevant chunks
        print("query to retriever \n", retrieval_query)
        docs = retrieve_chunks(vectorstore, retrieval_query, k=k)
        # 3. Combine retrieved content as context string
        context = "\n\n".join([doc.page_content for doc in docs])

        # 4. Build full LLM question
        question = format_question_for_batch(batch)

        # 5. Invoke the LLM chain
        try:
            ai_response = llm_chain.run({"context": context, "question": question})
        except Exception as e:
            print(f"[extract_fields_multi_step] LLM chain invocation failed: {e}")
            continue
        response_text = ai_response.strip()
        response_text = response_text.replace("`","")
        response_text = response_text.replace("json","")
        # 6. Parse JSON output
        try:
            parsed = json.loads(response_text)
            if not isinstance(parsed, dict):
                raise ValueError("Parsed JSON is not an object")
        except Exception as e:
            print(f"[extract_fields_multi_step] Warning: failed to parse JSON for batch {batch}: {e!r}")
            print(f"Response was: {response_text!r}")
            continue

        # 7. Merge into final_result; later batches override earlier for same key
        for key, val in parsed.items():
            final_result[key] = val

    return final_result




In [4]:
# if __name__ == "__main__":
# === User parameters ===
# PDF path configured at top (PDF_PATH)
# Batches: list of lists of field-info dicts
PDF_PATH = "pdfs/Losec Capsules 20mg (SmPC) -Chapter8-Example.pdf"  # modify as needed
batches = [
    [
        {
            "name": "Full Name",
            "description": "Extract the whole name from the section",
            "section": "1. Name of the medicinal product"
        },
        {
            "name": "Dosage Form",
            "description": "Pharmaceutical Dose Form Part, e.g., tablet, capsule, injection",
            "section": "1. Name of the medicinal product"
        },
        {
            "name": "Strength Part",
            "description": "Strength per unit, e.g., '500 mg'",
            "section": "1. Name of the medicinal product"
        }
    ],
    [
        {
            "name": "country",
            "description": "Extract full name of the Country or region for which this document applies",
            "section": "7. Marketing authorisation holder"
        },
        {
            "name": "language",
            "description": "Detect Language from the pdf text",
            "section": "" 
        }
    ]
]


# 1. Build vectorstore from PDF
print("Building FAISS vectorstore from PDF...")
vectorstore = build_vectorstore(PDF_PATH, chunk_size=1000, chunk_overlap=200)

# 2. Build LLMChain with customizable system prompt
llm_chain = build_llm_chain(llm)

# 3. Run multi-step extraction
print("Starting multi-step extraction...")
result = extract_fields_multi_step(
    vectorstore=vectorstore,
    batches=batches,
    llm_chain=llm_chain,
    k=2  # e.g., retrieve top-2 chunks per query
)

# 4. Print merged result as pretty JSON
print("Final extracted result:")
print(json.dumps(result, indent=2))


Building FAISS vectorstore from PDF...
[build_vectorstore_combined] Created 51 chunks after combining pages.
input_variables=['context', 'question'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a data extraction assistant. Output must be valid JSON ONLY.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Given the following extracted text from drug detail PDFs, answer the request below and return EXACTLY a JSON object with the keys specified (no extra keys).\n\n{context}\n\nRequest: {question}'), additional_kwargs={})]
Starting multi-step extraction...
query to retriever 
 Full Name in section '1. Name of the medicinal product' , Dosage Form in section '1. Name of the medicinal product' , Strength Part in section '1. Name of the medicinal product'
[r

C:\Users\thamilarasan.p\AppData\Local\Temp\ipykernel_14496\3414662033.py:83: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
C:\Users\thamilarasan.p\AppData\Local\Temp\ipykernel_14496\3414662033.py:48: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)
C:\Users\thamilarasan.p\AppData\Local\Temp\ipykernel_14496\3414662033.py:165: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_response = llm_chain.run({"context": context, "question": question})


query to retriever 
 country in section '7. Marketing authorisation holder' , language
[retrieve_chunks] Retrieved 2 chunks for query: "country in section '7. Marketing authorisation holder' , language"
--- Chunk 1 preview ---
6.6 Special precautions for disposal and other handling
No special requirements.
7. Marketing authorisation holder
Neon Healthcare Ltd.
8 The Chase, John Tate Road,
Hertford,
SG13 7NN
United Kingdom
8. Marketing authorisation number(s)
PL 45043/0101
9. Date of first authorisation/renewal of the authorisation
28/02/2011
10. Date of revision of the text
30.08.2023
Company Contact Details
Neon Healthcare Ltd...

--- Chunk 2 preview ---
+44 (0)1992 926 330
Stock Availability
+44 (0)1992 926 330...

Final extracted result:
{
  "full_name": "Losec 20 mg hard gastro-resistant capsules",
  "dosage_form": "gastro-resistant capsule, hard (gastro-resistant capsule)",
  "strength_part": "20 mg",
  "country": "United Kingdom",
  "language": "English"
}


In [ ]:
{
  "full_name": "Losec Control 20 mg gastro-resistant tablets",
  "dosage_form": "gastro-resistant tablet",
  "strength_part": "20 mg",
  "country": "Ireland",
  "language": "English"
}
{
  "full_name": "Losec 20 mg hard gastro-resistant capsules",
  "dosage_form": "gastro-resistant capsule, hard (gastro-resistant capsule)",
  "strength_part": "20 mg omeprazole",
  "country": "United Kingdom",
  "language": "English"
}

In [1]:
def make_class(message):
    class Greeter:
        def greet(self):
            return message  # captured from outer scope
    return Greeter

HiClass = make_class("Hi there!")
print(HiClass().greet())  # Hi there!

Hi there!


In [3]:
import requests

url = "https://spor.ema.europa.eu/v1/lists/200000000004/terms"
resp = requests.get(url)
print(resp)
data = resp.json()  # Contains all terms directly

for term in data['terms']:
    print(term['id'], term['prefLabel'][0]['value'])

<Response [401]>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thamilarasan.p\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [17]:
import os
from dotenv import load_dotenv

# Install these via pip:
# pip install pdfplumber langchain langchain_community ollama-client faiss-cpu rake-nltk

# --- Imports ---
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from rake_nltk import Rake

# === Configuration ===
load_dotenv()
BASE_URL = os.getenv("BASE_URL", "http://127.0.0.1:11434")
EMBED_MODEL = os.getenv("EMBED_MODEL", "maxbai-large")

# Ensure BASE_URL has a scheme:
if not BASE_URL.startswith(('http://', 'https://')):
    BASE_URL = 'http://' + BASE_URL

# --- Initialize Embeddings ---
embeddings = OllamaEmbeddings(
    model=EMBED_MODEL,
    base_url=BASE_URL,
)

# --- Functions ---
def load_and_split(pdf_path: str, chunk_size: int = 1000, chunk_overlap: int = 200):
    """
    Loads a PDF and splits into overlapping text chunks.
    """
    loader = PDFPlumberLoader(pdf_path)
    docs = loader.load()
    splitter = CharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return splitter.split_documents(docs)


def build_faiss_index(docs):
    """
    Builds a FAISS index over provided LangChain Documents.
    """
    return FAISS.from_documents(docs, embeddings)


def keyword_rerank(chunks, query: str, top_n: int = 5):
    """
    Re-ranks chunks by RAKE keyword overlap with the query.
    """
    rake = Rake()
    rake.extract_keywords_from_text(query)
    keywords = [kw.lower() for kw in rake.get_ranked_phrases()]

    scored = []
    for doc in chunks:
        text = doc.page_content.lower()
        score = sum(text.count(kw) for kw in keywords)
        scored.append((doc, score))

    scored.sort(key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in scored[:top_n]]


def search_pdf_with_ollama(
    pdf_path: str,
    query: str,
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
    top_k: int = 10,
    top_n: int = 10
):
    """
    Full pipeline: load PDF, build index, semantic search, and keyword re-rank.
    """
    # 1) Chunk the document
    docs = load_and_split(pdf_path, chunk_size, chunk_overlap)

    # 2) Build FAISS index
    vectorstore = build_faiss_index(docs)

    # 3) Semantic similarity search
    retrieved = vectorstore.similarity_search(query, k=top_k)

    # 4) Keyword-based re-ranking
    return keyword_rerank(retrieved, query, top_n)


if __name__ == "__main__":
    pdf_file = "pdfs/Losec Control SmPC.pdf"
    user_query = "Country in '7. MARKETING AUTHORISATION HOLDER' , Language"
    chunks = search_pdf_with_ollama(pdf_file, user_query)
    for i, chunk in enumerate(chunks, start=1):
        snippet = chunk.page_content[:200].replace("\n", " ")
        print(f"--- Chunk #{i} ---\n{snippet}\n")


--- Chunk #1 ---
6.3 Shelf life 3 years. 6.4 Special precautions for storage Do not store above 25°C. Store in the original package in order to protect from moisture. 6.5 Nature and contents of container Polyamide/Alu

--- Chunk #2 ---
Excretion The plasma elimination half-life of omeprazole is usually shorter than one hour both after single and repeated oral once-daily dosing. Omeprazole is completely eliminated from plasma between

--- Chunk #3 ---
Omeprazole should be discontinued in case of suspected TIN, and appropriate treatment should be promptly initiated. 4.5 Interaction with other medicinal products and other forms of interaction Effects

--- Chunk #4 ---
The inhibition of acid secretion is related to the area under the plasma concentration-time curve (AUC) of omeprazole and not to the actual plasma concentration at a given time. No tachyphylaxis has b

--- Chunk #5 ---
Very rare: Gynaecomastia General disorders and administration site conditions Uncommon: Malaise, peripheral

In [23]:
import os
from dotenv import load_dotenv

# Install these via pip:
# pip install pdfplumber langchain langchain_community ollama-client faiss-cpu rake-nltk

# --- Imports ---
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from rake_nltk import Rake

# === Configuration ===
load_dotenv()
BASE_URL = os.getenv("BASE_URL", "http://127.0.0.1:11434")
EMBED_MODEL = os.getenv("EMBED_MODEL", "maxbai-large")

# Ensure BASE_URL has a scheme:
if not BASE_URL.startswith(('http://', 'https://')):
    BASE_URL = 'http://' + BASE_URL

# --- Initialize Embeddings ---
embeddings = OllamaEmbeddings(
    model=EMBED_MODEL,
    base_url=BASE_URL,
)

# --- Functions ---

def load_and_split(pdf_path: str, chunk_size: int = 1000, chunk_overlap: int = 200):
    """
    Loads a PDF and splits it into overlapping text chunks.
    """
    loader = PDFPlumberLoader(pdf_path)
    docs = loader.load()
    splitter = CharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return splitter.split_documents(docs)


def build_faiss_index(docs):
    """
    Builds a FAISS index over provided LangChain Documents.
    """
    return FAISS.from_documents(docs, embeddings)


def rerank_with_weights(
    docs_with_scores,
    query: str,
    top_n: int = 5,
    alpha: float = 1.0,
    beta: float = 1.0
):
    """
    Re-ranks chunks by combining semantic score and weighted keyword overlap.

    Args:
        docs_with_scores: List of (doc, semantic_score) tuples.
        query: Original user query string.
        top_n: Number of top chunks to return.
        alpha: Weight for semantic similarity score.
        beta: Weight for keyword overlap score.

    Returns:
        List of top_n LangChain Documents.
    """
    # Extract RAKE keywords and their weights
    rake = Rake()
    rake.extract_keywords_from_text(query)
    scored_keys = [(score, phrase.lower()) for score, phrase in rake.get_ranked_phrases_with_scores()]

    combined_scores = []
    for doc, sem_score in docs_with_scores:
        text = doc.page_content.lower()
        kw_score = sum(text.count(kw) * weight for weight, kw in scored_keys)
        # Combine: higher sem_score and higher kw_score => higher final
        final_score = alpha * sem_score + beta * kw_score
        combined_scores.append((doc, final_score))

    # Sort by descending combined score
    combined_scores.sort(key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in combined_scores[:top_n]]


def search_pdf_with_ollama(
    pdf_path: str,
    query: str,
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
    top_k: int = 10,
    top_n: int = 5,
    alpha: float = 1.0,
    beta: float = 1.0
):
    """
    Full pipeline: load PDF, build index, semantic search, and combined re-rank.

    Args:
        pdf_path: Path to the PDF file.
        query: User query string.
        chunk_size, chunk_overlap: Params for text splitting.
        top_k: Number of chunks to retrieve semantically.
        top_n: Number of chunks to return after re-ranking.
        alpha: Weight for semantic similarity in ranking.
        beta: Weight for keyword match in ranking.
    """
    # 1) Chunk the document
    docs = load_and_split(pdf_path, chunk_size, chunk_overlap)

    # 2) Build FAISS index
    vectorstore = build_faiss_index(docs)

    # 3) Semantic similarity search with relevance scores
    docs_with_scores = vectorstore.similarity_search_with_relevance_scores(query, k=top_k)

    # 4) Combined re-ranking
    return rerank_with_weights(docs_with_scores, query, top_n, alpha, beta)


if __name__ == "__main__":
    pdf_file = "pdfs/Losec Control SmPC.pdf"
    user_query = "Shelf Life Time Period in '6. PHARMACEUTICAL PARTICULARS, 6.3 Shelf life' , Special Precautions for Storage  in '6.4 Special precautions for storage'"
    # Adjust alpha/beta to tune importance of semantics vs keywords
    chunks = search_pdf_with_ollama(
        pdf_file,
        user_query,
        top_k=15,
        top_n=5,
        alpha=1.0,
        beta=2.0
    )
    for i, chunk in enumerate(chunks, start=1):
        snippet = chunk.page_content[:200].replace("\n", " ")
        print(f"--- Chunk #{i} ---\n{snippet}\n")


--- Chunk #1 ---
Omeprazole should be discontinued in case of suspected TIN, and appropriate treatment should be promptly initiated. 4.5 Interaction with other medicinal products and other forms of interaction Effects

--- Chunk #2 ---
6.3 Shelf life 3 years. 6.4 Special precautions for storage Do not store above 25°C. Store in the original package in order to protect from moisture. 6.5 Nature and contents of container Polyamide/Alu

--- Chunk #3 ---
Summary of Product Characteristics 1. NAME OF THE MEDICINAL PRODUCT Losec Control 20 mg gastro-resistant tablets 2. QUALITATIVE AND QUANTITATIVE COMPOSITION Each gastro-resistant tablet contains 20.6 

--- Chunk #4 ---
The inhibition of acid secretion is related to the area under the plasma concentration-time curve (AUC) of omeprazole and not to the actual plasma concentration at a given time. No tachyphylaxis has b

--- Chunk #5 ---
4.3 Contraindications Hypersensitivity to omeprazole, substituted benzimidazoles or to any of the excipient

C:\Users\thamilarasan.p\AppData\Local\Temp\ipykernel_41904\980961642.py:118: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='8b0b325b-fa2c-4b69-967c-8f0ff2dd4c6e', metadata={'source': 'pdfs/Losec Control SmPC.pdf', 'file_path': 'pdfs/Losec Control SmPC.pdf', 'page': 2, 'total_pages': 9, 'Author': 'AIFA', 'Company': 'AIFA', 'CreationDate': "D:20230403125516+01'00'", 'Creator': 'Acrobat PDFMaker 22 for Word', 'Keywords': '', 'MSIP_Label_7f850223-87a8-40c3-9eb2-432606efca2a_ActionId': 'd3625aaa-fc16-402f-869b-64af1a766544', 'MSIP_Label_7f850223-87a8-40c3-9eb2-432606efca2a_ContentBits': '0', 'MSIP_Label_7f850223-87a8-40c3-9eb2-432606efca2a_Enabled': 'true', 'MSIP_Label_7f850223-87a8-40c3-9eb2-432606efca2a_Method': 'Privileged', 'MSIP_Label_7f850223-87a8-40c3-9eb2-432606efca2a_Name': '7f850223-87a8-40c3-9eb2-432606efca2a', 'MSIP_Label_7f850223-87a8-40c3-9eb2-432606efca2a_SetDate': '2022-12-21T18:27:43Z', 'MSIP_Label_7f850223-87a8-40c3-9eb2-432606efca2a_SiteId': 'fc

In [29]:
import os
from dotenv import load_dotenv

# Install these via pip:
# pip install pdfplumber langchain langchain_community openai faiss-cpu rake-nltk

# --- Imports ---
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from rake_nltk import Rake

# === Configuration ===
load_dotenv()
# Ensure OPENAI_API_KEY is set in your environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("Missing OPENAI_API_KEY in environment")

# --- Initialize Embeddings ---
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    openai_api_key=OPENAI_API_KEY
)

# --- Functions ---
def load_and_split(pdf_path: str, chunk_size: int = 1000, chunk_overlap: int = 200):
    """
    Loads a PDF and splits it into overlapping text chunks.
    """
    loader = PDFPlumberLoader(pdf_path)
    docs = loader.load()
    splitter = CharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return splitter.split_documents(docs)


def build_faiss_index(docs):
    """
    Builds a FAISS index over provided LangChain Documents.
    """
    return FAISS.from_documents(docs, embeddings)


def rerank_with_weights(
    docs_with_scores,
    query: str,
    top_n: int = 5,
    alpha: float = 1.0,
    beta: float = 1.0
):
    """
    Re-ranks chunks by combining semantic score and weighted keyword overlap.

    Args:
        docs_with_scores: List of (doc, semantic_score) tuples.
        query: Original user query string.
        top_n: Number of top chunks to return.
        alpha: Weight for semantic similarity score.
        beta: Weight for keyword overlap score.
    """
    rake = Rake()
    rake.extract_keywords_from_text(query)
    scored_keys = [(score, phrase.lower()) for score, phrase in rake.get_ranked_phrases_with_scores()]

    combined_scores = []
    for doc, sem_score in docs_with_scores:
        text = doc.page_content.lower()
        kw_score = sum(text.count(kw) * weight for weight, kw in scored_keys)
        final_score = alpha * sem_score + beta * kw_score
        combined_scores.append((doc, final_score))

    combined_scores.sort(key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in combined_scores[:top_n]]


def search_pdf(
    pdf_path: str,
    query: str,
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
    top_k: int = 10,
    top_n: int = 5,
    alpha: float = 1.0,
    beta: float = 1.0
):
    """
    Full pipeline: load PDF, build index, semantic search, and combined re-rank.
    """
    docs = load_and_split(pdf_path, chunk_size, chunk_overlap)
    vectorstore = build_faiss_index(docs)
    docs_with_scores = vectorstore.similarity_search_with_relevance_scores(query, k=top_k)
    return rerank_with_weights(docs_with_scores, query, top_n, alpha, beta)


if __name__ == "__main__":
    pdf_file = "pdfs/Losec Control SmPC.pdf"
    user_query = "Full Name in 1. NAME OF THE MEDICINAL PRODUCT , Invented Name Part in 1. NAME OF THE MEDICINAL PRODUCT"
    chunks = search_pdf(
        pdf_file,
        user_query,
        top_k=15,
        top_n=5,
        alpha=1.0,
        beta=2.0
    )
    for i, chunk in enumerate(chunks, start=1):
        snippet = chunk.page_content[:200].replace("\n", " ")
        print(f"--- Chunk #{i} ---\n{snippet}\n")


--- Chunk #1 ---
Monitoring phenytoin plasma concentration is recommended during the first two weeks after initiating omeprazole treatment and, if a phenytoin dose adjustment is made, monitoring and a further dose adj

--- Chunk #2 ---
Omeprazole should be discontinued in case of suspected TIN, and appropriate treatment should be promptly initiated. 4.5 Interaction with other medicinal products and other forms of interaction Effects

--- Chunk #3 ---
The inhibition of acid secretion is related to the area under the plasma concentration-time curve (AUC) of omeprazole and not to the actual plasma concentration at a given time. No tachyphylaxis has b

--- Chunk #4 ---
Very rare: Gynaecomastia General disorders and administration site conditions Uncommon: Malaise, peripheral oedema Rare: Increased sweating Reporting of suspected adverse reactions Reporting suspected

--- Chunk #5 ---
Summary of Product Characteristics 1. NAME OF THE MEDICINAL PRODUCT Losec Control 20 mg gastro-resistant ta

C:\Users\thamilarasan.p\AppData\Local\Temp\ipykernel_41904\58878669.py:95: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='10e30b10-c0f1-4df5-a3b7-c04d88241b03', metadata={'source': 'pdfs/Losec Control SmPC.pdf', 'file_path': 'pdfs/Losec Control SmPC.pdf', 'page': 8, 'total_pages': 9, 'Author': 'AIFA', 'Company': 'AIFA', 'CreationDate': "D:20230403125516+01'00'", 'Creator': 'Acrobat PDFMaker 22 for Word', 'Keywords': '', 'MSIP_Label_7f850223-87a8-40c3-9eb2-432606efca2a_ActionId': 'd3625aaa-fc16-402f-869b-64af1a766544', 'MSIP_Label_7f850223-87a8-40c3-9eb2-432606efca2a_ContentBits': '0', 'MSIP_Label_7f850223-87a8-40c3-9eb2-432606efca2a_Enabled': 'true', 'MSIP_Label_7f850223-87a8-40c3-9eb2-432606efca2a_Method': 'Privileged', 'MSIP_Label_7f850223-87a8-40c3-9eb2-432606efca2a_Name': '7f850223-87a8-40c3-9eb2-432606efca2a', 'MSIP_Label_7f850223-87a8-40c3-9eb2-432606efca2a_SetDate': '2022-12-21T18:27:43Z', 'MSIP_Label_7f850223-87a8-40c3-9eb2-432606efca2a_SiteId': 'fcb2

In [37]:
import os
from dotenv import load_dotenv

# Install these via pip:
# pip install pdfplumber langchain langchain_community rank-bm25

from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import CharacterTextSplitter
from rank_bm25 import BM25Okapi

# === Configuration ===
load_dotenv()

# --- Functions ---

def load_and_split(pdf_path: str, chunk_size: int = 1000, chunk_overlap: int = 200):
    """
    Loads a PDF and splits it into overlapping text chunks.
    Returns a list of LangChain Document objects.
    """
    loader = PDFPlumberLoader(pdf_path)
    docs = loader.load()
    splitter = CharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return splitter.split_documents(docs)


def build_bm25_index(docs):
    """
    Builds a BM25 index over the text content of the chunks.
    Returns the BM25 object and a list of tokenized chunks.
    """
    texts = [doc.page_content.lower() for doc in docs]
    tokenized = [text.split() for text in texts]
    bm25 = BM25Okapi(tokenized)
    return bm25, tokenized


def search_pdf_keyword(
    pdf_path: str,
    query: str,
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
    top_n: int = 5
):
    """
    Simple keyword-based retrieval using BM25.

    Args:
        pdf_path: path to PDF file
        query: user query string
        chunk_size: max characters per chunk
        chunk_overlap: overlapping characters between chunks
        top_n: number of chunks to return

    Returns:
        List of top_n LangChain Document objects ranked by BM25.
    """
    # 1) Load and split
    docs = load_and_split(pdf_path, chunk_size, chunk_overlap)

    # 2) Build BM25 index
    bm25, tokenized = build_bm25_index(docs)

    # 3) Tokenize query
    query_tokens = query.lower().split()

    # 4) Get BM25 scores
    scores = bm25.get_scores(query_tokens)

    # 5) Rank and select top_n
    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_n]
    return [docs[i] for i in top_indices]


if __name__ == "__main__":
    pdf_file = "pdfs/Losec 40 mg powder for solution for infusion ENG SmPC_09001bee8290ac93.pdf"
    user_query = "Full Name in '1. NAME OF THE MEDICINAL PRODUCT' , Invented Name Part in '1. NAME OF THE MEDICINAL PRODUCT' , Strength Part in '1. NAME OF THE MEDICINAL PRODUCT' , Pharmaceutical Dose Form Part in '1. NAME OF THE MEDICINAL PRODUCT'"
    chunks = search_pdf_keyword(
        pdf_file,
        user_query,
        chunk_size=1000,
        chunk_overlap=200,
        top_n=5
    )
    for i, chunk in enumerate(chunks, start=1):
        snippet = chunk.page_content.replace("\n", " ")
        print(f"--- Chunk #{i} ---\n{snippet}\n")


--- Chunk #1 ---
1. NAME OF THE MEDICINAL PRODUCT Losec 40 mg powder for solution for infusion 2. QUALITATIVE AND QUANTITATIVE COMPOSITION Each vial contains omeprazole sodium 42.6 mg, equivalent to omeprazole 40 mg. After reconstitution, 1 ml contains omeprazole sodium 0.426 mg, equivalent to omeprazole 0.4 mg. For the full list of excipients, see section 6.1. 3. PHARMACEUTICAL FORM Powder for solution for infusion (Powder for infusion) pH interval in glucose is approximately 8.9-9.5 and in sodium chloride 0.9%, 9.3-10.3 4. CLINICAL PARTICULARS 4.1 Therapeutic indications Losec for intravenous use is indicated in adults, as an alternative to oral therapy in: • Treatment of duodenal ulcers • Prevention of relapse of duodenal ulcers • Treatment of gastric ulcers • Prevention of relapse of gastric ulcers • In combination with appropriate antibiotics, Helicobacter pylori (H. pylori) eradication in peptic ulcer disease • Treatment of NSAID-associated gastric and duodenal ulcers • Preventio

In [38]:
import os
from dotenv import load_dotenv

# Install these via pip:
# pip install pdfplumber langchain langchain_community rank-bm25

from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import CharacterTextSplitter
from rank_bm25 import BM25Okapi

# === Configuration ===
load_dotenv()

# --- Functions ---

def load_and_split(pdf_path: str, chunk_size: int = 1000, chunk_overlap: int = 200):
    """
    Loads a PDF and splits it into overlapping text chunks.
    Returns a list of LangChain Document objects.
    """
    loader = PDFPlumberLoader(pdf_path)
    docs = loader.load()
    splitter = CharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return splitter.split_documents(docs)


def build_vector_store(
    docs,
):
    """
    Builds a BM25 vector store (index) from a list of LangChain Documents.
    Returns the BM25 object and the original docs.
    """
    # Lowercase and tokenize each document's content
    texts = [doc.page_content.lower() for doc in docs]
    tokenized = [text.split() for text in texts]
    bm25 = BM25Okapi(tokenized)
    return bm25


def retrieve_top_chunks(
    bm25: BM25Okapi,
    docs,
    query: str,
    top_n: int = 5
):
    """
    Retrieves the top_n document chunks for a given query using an existing BM25 index.

    Args:
        bm25: Pre-built BM25 index
        docs: List of LangChain Document objects used to build the index
        query: Query string
        top_n: Number of top chunks to return

    Returns:
        List of top_n LangChain Document objects ranked by relevance to the query
    """
    query_tokens = query.lower().split()
    scores = bm25.get_scores(query_tokens)
    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_n]
    return [docs[i] for i in top_indices]


if __name__ == "__main__":
    pdf_file = "pdfs/Losec 40 mg powder for solution for infusion ENG SmPC_09001bee8290ac93.pdf"
    # 1) Load and split into chunks
    docs = load_and_split(pdf_file, chunk_size=1000, chunk_overlap=200)
    
    # 2) Build the BM25 vector store
    bm25_index = build_vector_store(docs)

    # 3) Define your query and retrieve relevant chunks
    user_query = (
        "Full Name in '1. NAME OF THE MEDICINAL PRODUCT' , "
        "Invented Name Part in '1. NAME OF THE MEDICINAL PRODUCT' , "
        "Strength Part in '1. NAME OF THE MEDICINAL PRODUCT' , "
        "Pharmaceutical Dose Form Part in '1. NAME OF THE MEDICINAL PRODUCT'"
    )
    top_chunks = retrieve_top_chunks(bm25_index, docs, user_query, top_n=5)

    # 4) Print results
    for i, chunk in enumerate(top_chunks, start=1):
        snippet = chunk.page_content.replace("\n", " ")
        print(f"--- Chunk #{i} ---\n{snippet}\n")


--- Chunk #1 ---
1. NAME OF THE MEDICINAL PRODUCT Losec 40 mg powder for solution for infusion 2. QUALITATIVE AND QUANTITATIVE COMPOSITION Each vial contains omeprazole sodium 42.6 mg, equivalent to omeprazole 40 mg. After reconstitution, 1 ml contains omeprazole sodium 0.426 mg, equivalent to omeprazole 0.4 mg. For the full list of excipients, see section 6.1. 3. PHARMACEUTICAL FORM Powder for solution for infusion (Powder for infusion) pH interval in glucose is approximately 8.9-9.5 and in sodium chloride 0.9%, 9.3-10.3 4. CLINICAL PARTICULARS 4.1 Therapeutic indications Losec for intravenous use is indicated in adults, as an alternative to oral therapy in: • Treatment of duodenal ulcers • Prevention of relapse of duodenal ulcers • Treatment of gastric ulcers • Prevention of relapse of gastric ulcers • In combination with appropriate antibiotics, Helicobacter pylori (H. pylori) eradication in peptic ulcer disease • Treatment of NSAID-associated gastric and duodenal ulcers • Preventio

In [39]:
import re, pdfplumber

pattern = re.compile(r'^\d+(\.\d+)*\s+', re.MULTILINE)
chunks = []
with pdfplumber.open("pdfs/Losec Control SmPC.pdf") as pdf:
    full_text = ""
    for page in pdf.pages:
        full_text += (page.extract_text() or "") + "\n"
# Split by headings, keeping headings with content
matches = list(pattern.finditer(full_text))
for i, m in enumerate(matches):
    start = m.start()
    end = matches[i+1].start() if i+1 < len(matches) else len(full_text)
    title = m.group().strip()
    body = full_text[start+len(m.group()):end].strip()
    chunks.append({"heading": title, "content": body})
# Now chunks is a list of dicts, ready for JSON export.


In [57]:
import re
import fitz      # PyMuPDF
import pdfplumber

def inspect_font_sizes(pdf_path, max_samples=1000):
    size_map = {}
    doc = fitz.open(pdf_path)
    count = 0

    for page in doc:
        for block in page.get_text("dict")["blocks"]:
            if "lines" not in block:
                continue
            for line in block["lines"]:
                for span in line["spans"]:
                    text = span["text"].strip()
                    if not text:
                        continue
                    size = span["size"]
                    size_map.setdefault(size, []).append(text.replace("\n"," "))
                    count += 1
                    if count >= max_samples:
                        break
                if count >= max_samples:
                    break
            if count >= max_samples:
                break
        if count >= max_samples:
            break

    for size in sorted(size_map):
        samples = size_map[size][:5]
        print(f"Size {size:5.1f} — examples: {samples}")
    print("\nPick a threshold between the largest body size and smallest heading size.\n")


def chunk_with_font_regex(pdf_path, font_size_threshold):
    heading_re = re.compile(r'^\d+(\.\d+)*\b')
    chunks, current = [], {"heading": None, "content": []}

    doc = fitz.open(pdf_path)
    for page in doc:
        for block in page.get_text("dict")["blocks"]:
            if "lines" not in block:
                continue
            for line in block["lines"]:
                # reconstruct full line and its max span size
                line_text = "".join(span["text"] for span in line["spans"]).strip()
                max_size  = max((span["size"] for span in line["spans"]), default=0)

                is_heading = max_size >= font_size_threshold and heading_re.match(line_text)
                if is_heading:
                    # close out previous
                    if current["heading"]:
                        chunks.append({
                            "heading": current["heading"],
                            "content": "\n".join(current["content"]).strip()
                        })
                    current = {"heading": line_text, "content": []}
                else:
                    if current["heading"] and line_text:
                        current["content"].append(line_text)

    if current["heading"]:
        chunks.append({
            "heading": current["heading"],
            "content": "\n".join(current["content"]).strip()
        })
    return chunks


def chunk_with_pure_regex(pdf_path):
    # pull raw text (guarding None)
    full_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            txt = page.extract_text()
            if txt:
                full_text += txt + "\n"

    # find all heading matches
    heading_re = re.compile(r'(?m)^(?P<heading>\d+(\.\d+)*\b.*?)\r?$', re.MULTILINE)
    matches = list(heading_re.finditer(full_text))
    chunks = []
    for i, m in enumerate(matches):
        start = m.start()
        end   = matches[i+1].start() if i+1 < len(matches) else len(full_text)
        heading = m.group("heading").strip()
        body    = full_text[start+len(m.group(0)):end].strip()
        chunks.append({"heading": heading, "content": body})
    return chunks


In [60]:
pdf_file = "pdfs/Losec Control SmPC.pdf"

# Step 1: Inspect and choose threshold
print("=== Font size inspection ===")
inspect_font_sizes(pdf_file)
# e.g., you saw body ~9–11, headings ~10.5–12; choose threshold ~10.8
threshold = 11.0

# Step 2: Try font-size + regex chunking
sections = chunk_with_font_regex(pdf_file, font_size_threshold=threshold)

# Step 3: Fallback to pure-regex if empty
if not sections:
    print("Font-size logic gave no sections; falling back to pure-regex split.\n")
    sections = chunk_with_pure_regex(pdf_file)

# Display results
for sec in sections:
    print(f"\n--- {sec['heading']} ---\n length {len(sec['content'])} \n{sec['content'][:200]}...\n")

=== Font size inspection ===
Size   6.5 — examples: ['12', '12']
Size   7.0 — examples: ['max', '+', '+']
Size   9.0 — examples: ['BCH23004', '1/9', 'BCH23004', '2/9', 'BCH23004']
Size  10.5 — examples: ['1. NAME OF THE MEDICINAL PRODUCT', 'Losec Control 20 mg gastro-resistant tablets', '2. QUALITATIVE AND QUANTITATIVE COMPOSITION', 'Each gastro-resistant tablet contains 20.6 mg omeprazole magnesium equivalent to 20 mg omeprazole.', 'Excipients with known effect:']
Size  11.0 — examples: ['Methotrexate', 'SOC/frequency', 'Adverse reaction', 'Blood and lymphatic system disorders', 'Rare:']
Size  12.0 — examples: ['Summary of Product Characteristics']

Pick a threshold between the largest body size and smallest heading size.

Font-size logic gave no sections; falling back to pure-regex split.


--- 1. NAME OF THE MEDICINAL PRODUCT ---
 length 44 
Losec Control 20 mg gastro-resistant tablets...


--- 2. QUALITATIVE AND QUANTITATIVE COMPOSITION ---
 length 234 
Each gastro-resistant tablet

In [75]:
import re
import fitz  # PyMuPDF

def chunk_pdf_by_sections_with_bold(pdf_path, font_size_threshold=10.5):
    heading_re = re.compile(r'^\d+(\.\d+)*\b')
    chunks = []
    current = {"heading": None, "content": []}

    def is_bold_span(span):
        fname = span.get("font", "")
        flags = span.get("flags", 0)
        if any(sub in fname.lower() for sub in ("bold", "demi", "medium", "semi")):
            return True
        if flags & 2:
            return True
        return False

    doc = fitz.open(pdf_path)
    for page_num, page in enumerate(doc, start=1):
        for block in page.get_text("dict")["blocks"]:
            if "lines" not in block:
                continue
            for line in block["lines"]:
                spans = line["spans"]
                line_text = "".join(span["text"] for span in spans).strip()
                if not line_text:
                    continue

                sizes = [span["size"] for span in spans]
                max_size = max(sizes, default=0)
                bolds = [is_bold_span(span) for span in spans]
                has_bold = any(bolds)

                # DEBUG OUTPUT
                print(f"[Page {page_num}] '{line_text}' → size={max_size}, bold={has_bold}")

                is_heading = (
                    heading_re.match(line_text)
                    and max_size >= font_size_threshold
                    and has_bold
                )
                if is_heading:
                    print("  ➤ **HEADING**\n")
                    if current["heading"] is not None:
                        chunks.append({
                            "heading": current["heading"],
                            "content": "\n".join(current["content"]).strip()
                        })
                    current = {"heading": line_text, "content": []}
                else:
                    if current["heading"]:
                        current["content"].append(line_text)

    if current["heading"] is not None:
        chunks.append({
            "heading": current["heading"],
            "content": "\n".join(current["content"]).strip()
        })

    return chunks
    

In [77]:
pdf_file = "pdfs/Losec Control SmPC.pdf"

sections = chunk_pdf_by_sections_with_bold(
    "pdfs/Losec Control SmPC.pdf",
    font_size_threshold=12.0  # adjust after you inspect the debug output
)
for sec in sections:
    print(f"\n--- {sec['heading']} ---\n{sec['content'][:200]}...\n")


[Page 1] 'BCH23004' → size=9.0, bold=False
[Page 1] '1/9' → size=9.0, bold=False
[Page 1] 'Summary of Product Characteristics' → size=12.0, bold=True
[Page 1] '1. NAME OF THE MEDICINAL PRODUCT' → size=10.5, bold=True
[Page 1] 'Losec Control 20 mg gastro-resistant tablets' → size=10.5, bold=False
[Page 1] '2. QUALITATIVE AND QUANTITATIVE COMPOSITION' → size=10.5, bold=True
[Page 1] 'Each gastro-resistant tablet contains 20.6 mg omeprazole magnesium equivalent to 20 mg omeprazole.' → size=10.5, bold=False
[Page 1] 'Excipients with known effect:' → size=10.5, bold=False
[Page 1] 'Each gastro-resistant tablet contains 19–20 mg sucrose.' → size=10.5, bold=False
[Page 1] 'For the full list of excipients, see section 6.1.' → size=10.5, bold=False
[Page 1] '3. PHARMACEUTICAL FORM' → size=10.5, bold=True
[Page 1] 'Gastro-resistant tablet.' → size=10.5, bold=False
[Page 1] 'Pink, oblong, biconvex, film-coated tablets engraved with' → size=10.5, bold=False
[Page 1] 'on one side and 20 mg on the o

In [122]:
import re
import fitz  # PyMuPDF

def chunk_pdf_by_sections_with_bold(
    pdf_path,
    font_size_threshold=8.0,   # ← tuned down
    debug=False
):
    heading_re = re.compile(r'^\d+(\.\d+)*\b')
    chunks = []
    current = {"heading": None, "content": []}

    doc = fitz.open(pdf_path)
    for page_num, page in enumerate(doc, start=1):
        for block in page.get_text("dict")["blocks"]:
            if "lines" not in block:
                continue
            # print("hi ehllo")
            # print(block)
            for line in block["lines"]:
                line_text = "".join(span["text"] for span in line["spans"]).strip()
                if not line_text:
                    continue

                sizes = [span["size"] for span in line["spans"]]
                max_size = max(sizes)
                # look for "-Bold" or "_Bd" in the font name
                has_bold = any(
                    re.search(r'[-_]Bold', span.get("font", ""), re.IGNORECASE)
                    for span in line["spans"]
                )

                is_heading = (
                    heading_re.match(line_text)
                    and max_size >= font_size_threshold
                    and has_bold
                )

                if debug and (heading_re.match(line_text) or has_bold):
                    print(
                        f"[DEBUG] Pg {page_num:>2} | '{line_text[:30]}…' | "
                        f"max_size={max_size:.1f} | bold={has_bold} | heading? {is_heading}"
                    )

                if is_heading:
                    # close out the previous section
                    if current["heading"] is not None:
                        chunks.append({
                            "heading": current["heading"],
                            "content": "\n".join(current["content"]).strip()
                        })
                    # start a new section
                    current = {"heading": line_text, "content": []}
                else:
                    # collect body text
                    if current["heading"]:
                        current["content"].append(line_text)

    # append the last one
    if current["heading"] is not None:
        chunks.append({
            "heading": current["heading"],
            "content": "\n".join(current["content"]).strip()
        })

    doc.close()
    return chunks

# if __name__ == "__main__":
sections = chunk_pdf_by_sections_with_bold(
    "pdfs/Losec Control SmPC.pdf",
    font_size_threshold=9.6,
    debug=False
)
print(f"\nFound {len(sections)} sections.\n")
for sec in sections:
    print(f"--- {sec['heading']} ---\nlength = {len(sec['content'])}\n{sec['content'][:100]}...\n")



Found 28 sections.

--- 1. NAME OF THE MEDICINAL PRODUCT ---
length = 44
Losec Control 20 mg gastro-resistant tablets...

--- 2. QUALITATIVE AND QUANTITATIVE COMPOSITION ---
length = 234
Each gastro-resistant tablet contains 20.6 mg omeprazole magnesium equivalent to 20 mg omeprazole.
E...

--- 3. PHARMACEUTICAL FORM ---
length = 158
Gastro-resistant tablet.
Pink, oblong, biconvex, film-coated tablets engraved with
on one side and 2...

--- 4. CLINICAL PARTICULARS ---
length = 0
...

--- 4.1 Therapeutic indications ---
length = 136
Losec Control gastro-resistant tablets is indicated for the treatment of reflux symptoms (e.g. heart...

--- 4.2 Posology and method of administration ---
length = 1347
Posology in adults
The recommended dose is 20 mg once daily for 14 days.
It might be necessary to ta...

--- 4.3 Contraindications ---
length = 220
Hypersensitivity to omeprazole, substituted benzimidazoles or to any of the excipients listed in sec...

--- 4.4 Special warnings and precaution

In [133]:
import re
from typing import List, Dict

def retrieve_section_text(
    chunks: List[Dict[str, str]],
    sections: List[str]
) -> str:
    """
    Retrieve combined text for the given sections.
    
    - Purely numeric section keys (e.g. "1.", "2.3") match by numeric prefix.
    - Any key containing letters matches by substring of the heading.
    """
    # Pre-compile regexes
    num_re    = re.compile(r'^(\d+(?:\.\d+)*)')         # to extract heading prefix
    pure_num  = re.compile(r'^\d+(?:\.\d+)*\.?$')       # to recognize "1." or "2.3"
    
    # Classify requests
    numeric_only = set()
    textual_only = []
    for sec in sections:
        s = sec.strip()
        if pure_num.match(s):
            # normalize "1." -> "1"
            numeric_only.add(s.rstrip('.'))
        else:
            textual_only.append(s.lower())

    collected = []
    seen = set()

    for chunk in chunks:
        heading = chunk["heading"].strip()
        heading_lower = heading.lower()

        # Extract numeric prefix if it exists
        m = num_re.match(heading)
        num_prefix = m.group(1) if m else None

        matched = False

        # 1) try numeric-only matches
        if num_prefix:
            for num in numeric_only:
                if num_prefix == num or num_prefix.startswith(num + "."):
                    matched = True
                    break

        # 2) if not numeric-matched, try textual matches
        if not matched:
            for txt in textual_only:
                if txt in heading_lower:
                    matched = True
                    break

        # 3) collect content if matched
        if matched:
            content = chunk["content"]
            if content and content not in seen:
                seen.add(content)
                collected.append(content)

    # Join with blank line separators
    return "\n\n".join(collected)


In [139]:
chunks = [
    {"heading": "1.1 Introduction", "content": "Welcome…"},
    {"heading": "1.2 Overview",     "content": "This section covers…"},
    {"heading": "1.2.1 Details",     "content": "Deep dive…"},
    {"heading": "2.1 Next",         "content": "Another…"},
]

print(retrieve_section_text(sections, [""]))

Losec Control 20 mg gastro-resistant tablets

Each gastro-resistant tablet contains 20.6 mg omeprazole magnesium equivalent to 20 mg omeprazole.
Excipients with known effect:
Each gastro-resistant tablet contains 19–20 mg sucrose.
For the full list of excipients, see section 6.1.

Gastro-resistant tablet.
Pink, oblong, biconvex, film-coated tablets engraved with
on one side and 20 mg on the other side,
containing enteric coated pellets.

Losec Control gastro-resistant tablets is indicated for the treatment of reflux symptoms (e.g. heartburn, acid
regurgitation) in adults.

Posology in adults
The recommended dose is 20 mg once daily for 14 days.
It might be necessary to take the tablets for 2-3 consecutive days to achieve improvement of symptoms.
The majority of patients achieve complete relief of heartburn within 7 days. Once complete relief of symptoms
has occurred, treatment should be discontinued.
Special populations
Impaired renal function
Dose adjustment is not needed in patients 

In [137]:
sections

[{'heading': '1. NAME OF THE MEDICINAL PRODUCT',
  'content': 'Losec Control 20 mg gastro-resistant tablets'},
 {'heading': '2. QUALITATIVE AND QUANTITATIVE COMPOSITION',
  'content': 'Each gastro-resistant tablet contains 20.6 mg omeprazole magnesium equivalent to 20 mg omeprazole.\nExcipients with known effect:\nEach gastro-resistant tablet contains 19–20 mg sucrose.\nFor the full list of excipients, see section 6.1.'},
 {'heading': '3. PHARMACEUTICAL FORM',
  'content': 'Gastro-resistant tablet.\nPink, oblong, biconvex, film-coated tablets engraved with\non one side and 20 mg on the other side,\ncontaining enteric coated pellets.'},
 {'heading': '4. CLINICAL PARTICULARS', 'content': ''},
 {'heading': '4.1 Therapeutic indications',
  'content': 'Losec Control gastro-resistant tablets is indicated for the treatment of reflux symptoms (e.g. heartburn, acid\nregurgitation) in adults.'},
 {'heading': '4.2 Posology and method of administration',
  'content': 'Posology in adults\nThe recom

In [74]:
import fitz # PyMuPDF

def extract_text_with_font_size(pdf_path):
    doc = fitz.open(pdf_path)
    for page_num in range(len(doc)):
        page = doc[page_num]
        text_dict = page.get_text("dict")
        for block in text_dict["blocks"]:
            if "lines" in block:
                for line in block["lines"]:
                    line_text = ""
                    font_sizes = set()
                    for span in line["spans"]:
                        line_text += span["text"]
                        font_sizes.add(span["size"])
                    print(f"Page {page_num + 1}, Line: '{line_text.strip()}', Font Sizes: {list(font_sizes)}")
    doc.close()

# Example usage:
extract_text_with_font_size("pdfs/Losec Control SmPC.pdf")

Page 1, Line: 'BCH23004', Font Sizes: [9.0]
Page 1, Line: '1/9', Font Sizes: [9.0]
Page 1, Line: '', Font Sizes: [9.0]
Page 1, Line: '', Font Sizes: [7.980000019073486]
Page 1, Line: 'Summary of Product Characteristics', Font Sizes: [12.0]
Page 1, Line: '', Font Sizes: [10.979999542236328]
Page 1, Line: '1. NAME OF THE MEDICINAL PRODUCT', Font Sizes: [10.5]
Page 1, Line: '', Font Sizes: [10.5]
Page 1, Line: 'Losec Control 20 mg gastro-resistant tablets', Font Sizes: [10.5]
Page 1, Line: '', Font Sizes: [10.5]
Page 1, Line: '2. QUALITATIVE AND QUANTITATIVE COMPOSITION', Font Sizes: [10.5]
Page 1, Line: '', Font Sizes: [10.5]
Page 1, Line: 'Each gastro-resistant tablet contains 20.6 mg omeprazole magnesium equivalent to 20 mg omeprazole.', Font Sizes: [10.5]
Page 1, Line: 'Excipients with known effect:', Font Sizes: [10.5]
Page 1, Line: 'Each gastro-resistant tablet contains 19–20 mg sucrose.', Font Sizes: [10.5]
Page 1, Line: '', Font Sizes: [10.5]
Page 1, Line: 'For the full list of ex